<h1 style="color:red"; font-size:30px">KNOWLEDGE EXTRACTION</h1>

<p>The main goal of our project is to make a <b>comparison</b>, from a semantic point of view, between the <b>Fabrizio De André music album</b>, with the title <b>"Non al denaro non all'amore né al cielo"</b>, and the <b>Edgar Lee Masters' Spoon River anthology poems</b>.</p> 
<p>Fabrizio De André for its fifth album, published in 1971, decided to give a personal interpretation of nine of the Spoon River's poems, developing in particular the <b>envy</b> and the <b>science</b> topics.<br>
During an interview made by Fernanda Pivano, who translated and reviewed together with Cesare Pavese the first italian edition of the Spoon River's anthology, Fabrizio De André said:<br>
   <em>“Spoon River l’ho letto da ragazzo, avrò avuto 18 anni. Mi era piaciuto [...] forse perché in questi personaggi si trova qualcosa di me. Poi mi è capitato di rileggerlo, due anni fa [1969, ndr], e mi sono reso conto che non era invecchiato per niente. Soprattutto mi ha colpito un fatto: nella vita, si è costretti alla competizione, magari si è costretti a pensare il falso o a non essere sinceri, nella morte, invece, i personaggi di Spoon River si esprimono con estrema sincerità, perché non hanno più da aspettarsi niente, non hanno più niente da pensare. Così parlano come da vivi non sono mai stati capaci di fare”.</em>
<p>Our main research questions are: <br>
<b><em>"Is it possible to extract in an automatic way the main themes treated in each of the nine Spoon River poems and in the De André corresponding songs?"<br>
"What are the differences between them?" <br> "Are the results that we gather from this topic extraction in line with the personal interpretation of De André about his own music album?"</em></b></p>

<h2 style="color:red">Methodology</h2>

<p>To accomplish this knowledge extraction task we decided to use <b><a href="https://maartengr.github.io/BERTopic/index.html">BERTopic</a></b>, which is a a topic modeling technique that leverages BERT embeddings and <a href="https://maartengr.github.io/BERTopic/api/ctfidf.html">c-TF-IDF</a> to create dense clusters allowing for easily interpretable topics whilst keeping important words in the topic descriptions.</p>

<p>We decided to use as corpus for our research the entire Spoon River anthology, consisting of 244 poems in the last 1916 edition, and due to its reduced dimension we found more interesting to use an <a href="https://maartengr.github.io/BERTopic/getting_started/embeddings/embeddings.html">embedding model</a>, that allowed us to leverage pre-trained embeddings suitable for our use case, and to create topics, even if we had little data available.</p>
<p>We used a general purpose trained model, <b><a href="https://www.sbert.net/docs/pretrained_models.html">all-MiniLM-L6-v2</a></b> in particular, which has been trained on a large and diverse dataset of over 1 billion training pairs and offers good quality and high speed.</p>

In [2]:
import numpy as np
import pandas as pd
from copy import deepcopy
from bertopic import BERTopic

from bertopic import BERTopic
topic_model = BERTopic(embedding_model="all-MiniLM-L6-v2")

Then through the method <a href="https://maartengr.github.io/BERTopic/api/bertopic.html#bertopic._bertopic.BERTopic.fit"><code>  fit(documents, embeddings=None, y=None)</code></a> we fitted the trained model on our collection of documents, formed by not the entire poems but single verses. <br> Before using the method <code> fit() </code>, we have pre-processed our corpus removing the punctuation and changing all the characters to lowercase ones through the following functions:

In [9]:
#in this string we put the texts of all the Spoon River Anthology poems

poems='''
  The Hill
  Where are Elmer, Herman, Bert, Tom and Charley,
  The weak of will, the strong of arm, the clown, the boozer, the fighter?
  All, all are sleeping on the hill.
  One passed in a fever,
  One was burned in a mine,
  One was killed in a brawl,
  One died in a jail,
  One fell from a bridge toiling for children and wife--
  All, all are sleeping, sleeping, sleeping on the hill.
  Where are Ella, Kate, Mag, Lizzie and Edith,
  The tender heart, the simple soul, the loud, the proud, the happy one?--
  All, all are sleeping on the hill.
  One died in shameful child-birth,
  One of a thwarted love,
  One at the hands of a brute in a brothel,
  One of a broken pride, in the search for heart's desire;
  One after life in far-away London and Paris
  Was brought to her little space by Ella and Kate and Mag--
  All, all are sleeping, sleeping, sleeping on the hill.
  Where are Uncle Isaac and Aunt Emily,
  And old Towny Kincaid and Sevigne Houghton,
  And Major Walker who had talked
  With venerable men of the revolution?--
  All, all are sleeping on the hill.
  They brought them dead sons from the war,
  And daughters whom life had crushed,
  And their children fatherless, crying--
  All, all are sleeping, sleeping, sleeping on the hill.
  Where is Old Fiddler Jones
  Who played with life all his ninety years,
  Braving the sleet with bared breast,
  Drinking, rioting, thinking neither of wife nor kin,
  Nor gold, nor love, nor heaven?
  Lo! he babbles of the fish-frys of long ago,
  Of the horse-races of long ago at Clary's Grove,
  Of what Abe Lincoln said
  One time at Springfield.
  Hod Putt
  HERE I lie close to the grave
  Of Old Bill Piersol,
  (...)'''


In [10]:
#through a simple split we obtain a list of strings, in which each string is a verse.
verses = poems.split("\n")

In [13]:
import nltk, nltk.tokenize, nltk.corpus, nltk.stem
from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords, wordnet

#lowercase text
def to_lower(text):
    return ' '.join([w.lower() for w in word_tokenize(text)])

#remove stopwords
def remove_stopwords(text):
    no_stop = text.split()
    stopwordz = stopwords.words('english')
    for word in no_stop:
        for stopword in stopwordz:
            if word == stopword:
                no_stop.remove(word)
    return ' '.join(no_stop)

#remove punctuation
def strip_punctuation(text):
    tokenizer = RegexpTokenizer(r'\w+')
    return ' '.join(tokenizer.tokenize(text))

#preprocess text
def preprocess(text):
    lower_text = to_lower(text)
    text_no_stopwords = remove_stopwords(lower_text)
    text_no_punct = strip_punctuation(text_no_stopwords)
    return text_no_punct

#application of the function preprocess(text) to our corpus 
verses_no_punct=[]
for x in verses:
    x_preprocessed=preprocess(x)
    verses_no_punct.append(x_preprocessed)

<p>Then we passed as parameter of the method <code> fit() </code> the list of pre-processed strings containing the verses of the Spoon River poems.</p>

In [ ]:
topic_model.fit(verses_no_punct)

<p>Using the method<a href="https://maartengr.github.io/BERTopic/api/bertopic.html#bertopic._bertopic.BERTopic.get_topic_freq"> <code>topic_model.get_topic_freq(topic=none)</code></a> is possible to obtain the number and the frequence of all the retrieved topics.<br> We have found 88 different topics for the entire Spoon River anthology. It is important to point out that the <code>topic -1</code> represents always the outliers so it can be ignored.</p>

In [9]:
topic_model.get_topic_freq()

,Topic,Count
0,-1,1616
1,0,118
2,1,114
3,2,103
4,3,92
...,...,...
84,83,11
85,84,11
86,85,10
87,86,10


<p>Our intuition has been to use the method <a href="https://maartengr.github.io/BERTopic/api/bertopic.html#bertopic._bertopic.BERTopic.find_topics"><code>find_topics(search_term, top_n=5)</code></a>using as parameter an entire poem or an entire song, in order to see which of the topics retrieved for the entire Spoon River anthology would have been associated to that specific poem or song.<br>
Furthermore the method <a href="https://maartengr.github.io/BERTopic/api/bertopic.html#bertopic._bertopic.BERTopic.get_topic"><code>get_topic(topic)</code></a> returns top n words for a specific topic.</p> 
<p>We have used this method for all the nine songs of the music album "Non al denaro non all'amore né al cielo" and for the nine poems of the Spoon River anthology from which De André took inspiration.<br>
In the following cells it is possible to check the results that we obtained.</p>

<h2 style="color:red">Results </h2>

<h4 style="color:red">1a - La collina</h4>

In [22]:
topic_model.find_topics("""They sleep on the hill Where has Elmer gone he, who let himself die of fever where Herman who was burned in a mine cave. Where are Bert and Tom the first killed in a brawl and the other who came out of prision already dead. What will be of Charley who fell while working from the bridge flew, and flew on the road. They sleep, they sleep on the hill they sleep, they sleep on the hill. Where are Ella and Kate both died by mistake one of abortion, the other of love. And Maggie killed in a brothel by the caresses of an animal and Edith and consumed by a strange evil. And Lizzie, who pursued life far away, and from England she was brought back in this small piece of land. They sleep, they sleep on the hill they sleep, they sleep on the hill. Where are the Generals who were honored in the battles with cemeteries of crosses on their chests, where the children of the war who have left for an ideal for a scam, for a badly ended love, they have sent home their remains in the flags tightly tied so that they could still seem integer. They sleep, they sleep on the hill they sleep, they sleep on the hill. Where is Jones the player who was surprised by his ninety years and who would have still played with life. He, who offered his face to the wind the throat to the wine and never a thought not money, not to heaven nor to love. He, it still seems possible to hear him chatting about dirty things eat on the street in the wrong hours, it seems again possible to hear him tell the merchant of liquors "You, who sell it, what better thing do you buy for yourself?""")

([25, 69, 43, 2, 5],
 [0.367177523713194,
  0.3585694716629595,
  0.3569291820125505,
  0.29981799710945534,
  0.2573903827852931])

In [23]:
topic_model.get_topic(25)

[('hill', 0.13118030038898457),
 ('hills', 0.1300899616858994),
 ('mountains', 0.11492382712001638),
 ('mountain', 0.08619287034001229),
 ('river', 0.07524063020438908),
 ('slopes', 0.06730202658370817),
 ('valleys', 0.06730202658370817),
 ('road', 0.04960744456542515),
 ('golden', 0.04612162308684979),
 ('fields', 0.044670311920375155)]

In [24]:
topic_model.get_topic(69)

[('sleeping', 0.4966235898801303),
 ('sleep', 0.31495806882989147),
 ('hill', 0.22651008390147648),
 ('are', 0.17243720104450572),
 ('side', 0.10675244260920477),
 ('maternal', 0.07776776833365001),
 ('ancestors', 0.07776776833365001),
 ('pekin', 0.07776776833365001),
 ('cells', 0.07776776833365001),
 ('existent', 0.07776776833365001)]

In [25]:
topic_model.get_topic(43)

[('grave', 0.32061178500602866),
 ('graven', 0.09952108186314292),
 ('cemetery', 0.09952108186314292),
 ('funeral', 0.09952108186314292),
 ('buried', 0.09099289614927777),
 ('close', 0.0660550357120441),
 ('ach', 0.05708485122363671),
 ('graves', 0.05708485122363671),
 ('lays', 0.05708485122363671),
 ('denied', 0.05708485122363671)]

In [26]:
topic_model.get_topic(2)

[('ernest', 0.05872945577089072),
 ('hyde', 0.04716911160679473),
 ('tom', 0.04716911160679473),
 ('roy', 0.04716911160679473),
 ('hamilton', 0.04716911160679473),
 ('john', 0.046912609423265236),
 ('willie', 0.04163665974656238),
 ('george', 0.036603542204094675),
 ('enoch', 0.034393315055644975),
 ('dunlap', 0.034393315055644975)]

In [27]:
topic_model.get_topic(5)

[('mrs', 0.12549066161559208),
 ('emily', 0.10871180982468169),
 ('daisy', 0.09722465569157962),
 ('fraser', 0.08969516648210586),
 ('sparks', 0.08502380969267992),
 ('williams', 0.07195681067424449),
 ('mary', 0.05979677765473724),
 ('mcneely', 0.046590775639149304),
 ('armstrong', 0.042139557185294746),
 ('hannah', 0.042139557185294746)]

<h4 style=" color:red">1b - The hill</h4>

In [18]:
topic_model.find_topics("""The Hill Whe re are Elmer, Herman, Bert, Tom and Charley, The weak of will, the strong of arm, the clown, the boozer, the fighter? All, all are sleeping on the hill. One passed in a fever, One was burned in a mine, One was killed in a brawl, One died in a jail, One fell from a bridge toiling for children and wife — All, all are sleeping, sleeping, sleeping on the hill. Where are Ella, Kate, Mag, Lizzie and Edith, The tender heart, the simple soul, the loud, the proud, the happy one? — All, all are sleeping on the hill. One died in shameful child-birth, One of a thwarted love, One at the hands of a brute in a brothel, One of a broken pride, in the search for heart's desire, One after life in far-away London and Paris Was brought to her little space by Ella and Kate and Mag — All, all are sleeping, sleeping, sleeping on the hill. Where are Uncle Isaac and Aunt Emily, And old Towny Kincaid and Sevigne Houghton, And Major Walker who had talked With venerable men of the revolution? — All, all are sleeping on the hill. They brought them dead sons from the war, And daughters whom life had crushed, And their children fatherless, crying — All, all are sleeping, sleeping, sleeping on the hill. Where is Old Fiddler Jones Who played with life all his ninety years, Braving the sleet with bared breast, Drinking, rioting, thinking neither of wife nor kin, Nor gold, nor love, nor heaven? Lo! he babbles of the fish-frys of long ago, Of the horse-races of long ago at Clary's Grove, Of what Abe Lincoln said One time at Springfield.""")

([25, 69, 43, 2, 21],
 [0.3281639032251219,
  0.3017498246450462,
  0.26493756191017714,
  0.22501265206921192,
  0.1962193752340611])

In [19]:
topic_model.get_topic(25)

[('hill', 0.13118030038898457),
 ('hills', 0.1300899616858994),
 ('mountains', 0.11492382712001638),
 ('mountain', 0.08619287034001229),
 ('river', 0.07524063020438908),
 ('slopes', 0.06730202658370817),
 ('valleys', 0.06730202658370817),
 ('road', 0.04960744456542515),
 ('golden', 0.04612162308684979),
 ('fields', 0.044670311920375155)]

In [20]:
topic_model.get_topic(69)

[('sleeping', 0.4966235898801303),
 ('sleep', 0.31495806882989147),
 ('hill', 0.22651008390147648),
 ('are', 0.17243720104450572),
 ('side', 0.10675244260920477),
 ('maternal', 0.07776776833365001),
 ('ancestors', 0.07776776833365001),
 ('pekin', 0.07776776833365001),
 ('cells', 0.07776776833365001),
 ('existent', 0.07776776833365001)]

In [21]:
topic_model.get_topic(43)

[('grave', 0.32061178500602866),
 ('graven', 0.09952108186314292),
 ('cemetery', 0.09952108186314292),
 ('funeral', 0.09952108186314292),
 ('buried', 0.09099289614927777),
 ('close', 0.0660550357120441),
 ('ach', 0.05708485122363671),
 ('graves', 0.05708485122363671),
 ('lays', 0.05708485122363671),
 ('denied', 0.05708485122363671)]

In [28]:
topic_model.get_topic(2)

[('ernest', 0.05872945577089072),
 ('hyde', 0.04716911160679473),
 ('tom', 0.04716911160679473),
 ('roy', 0.04716911160679473),
 ('hamilton', 0.04716911160679473),
 ('john', 0.046912609423265236),
 ('willie', 0.04163665974656238),
 ('george', 0.036603542204094675),
 ('enoch', 0.034393315055644975),
 ('dunlap', 0.034393315055644975)]

In [29]:
topic_model.get_topic(21)

[('village', 0.31640243125569956),
 ('town', 0.07049988155057592),
 ('galilee', 0.04847140774681572),
 ('atheist', 0.04847140774681572),
 ('really', 0.033217127508145705),
 ('in', 0.02987682984030964),
 ('dreariness', 0.027802984533791975),
 ('weaver', 0.027802984533791975),
 ('merriment', 0.027802984533791975),
 ('appearing', 0.027802984533791975)]

<h4 style="color:red">2a - Un matto</h4>

In [30]:
topic_model.find_topics("""A madman(behind every idiot there is a village) Try and have a world in your heart and you can’t express it in words, and daylight divides the piazza between a village that laughs at you, the idiot, who passes, and not even the night leaves you alone: the others dream about themselves and you dream about them. And yes, even you would go and seek the words that are certain to make you heard: to amaze for half an hour all you need is a history book, and I tried to learn the encyclopedia by heart, and after mackerel, mackintosh and maculate (1), the others continued, until reading mad. And without knowing to whom I owed my life I gave it back in a mental hospital: here on the hill I sleep unwillingly and yet now there’s light in my thoughts, here in the semi-darkness I now invent words but I miss a light, the light of the sun. My bones still give to life: they still give it flowery grass. But life has remained in the mute voices of those who lost the idiot and who cry for him on the hill of those who still whisper with the same irony: “A pitiful death snatched him from madness".""")

([65, 48, 21, 43, 17],
 [0.29533787186261506,
  0.26303399699793106,
  0.25293205877252384,
  0.2501332931492248,
  0.24855019463792818])

In [31]:
topic_model.get_topic(65)

[('dreams', 0.4298658477876101),
 ('dream', 0.16602979664254366),
 ('dreamed', 0.14392279530977592),
 ('pageant', 0.08255347715418233),
 ('abruptly', 0.08255347715418233),
 ('mirrored', 0.08255347715418233),
 ('glassed', 0.08255347715418233),
 ('slept', 0.08255347715418233),
 ('borne', 0.08255347715418233),
 ('leafy', 0.07196139765488796)]

In [32]:
topic_model.get_topic(48)

[('memory', 0.2528549455114603),
 ('memories', 0.19701242270268846),
 ('forget', 0.08909506376319462),
 ('page', 0.08146030702887726),
 ('stand', 0.07959693393903268),
 ('forgotten', 0.06852023353501803),
 ('remember', 0.06321373637786508),
 ('shines', 0.05110453347639859),
 ('urns', 0.05110453347639859),
 ('rupturing', 0.05110453347639859)]

In [33]:
topic_model.get_topic(21)

[('village', 0.31640243125569956),
 ('town', 0.07049988155057592),
 ('galilee', 0.04847140774681572),
 ('atheist', 0.04847140774681572),
 ('really', 0.033217127508145705),
 ('in', 0.02987682984030964),
 ('dreariness', 0.027802984533791975),
 ('weaver', 0.027802984533791975),
 ('merriment', 0.027802984533791975),
 ('appearing', 0.027802984533791975)]

In [34]:
topic_model.get_topic(43)

[('grave', 0.32061178500602866),
 ('graven', 0.09952108186314292),
 ('cemetery', 0.09952108186314292),
 ('funeral', 0.09952108186314292),
 ('buried', 0.09099289614927777),
 ('close', 0.0660550357120441),
 ('ach', 0.05708485122363671),
 ('graves', 0.05708485122363671),
 ('lays', 0.05708485122363671),
 ('denied', 0.05708485122363671)]

In [35]:
topic_model.get_topic(17)

[('write', 0.13361456655916268),
 ('read', 0.12571654939042506),
 ('books', 0.11053285532882966),
 ('letter', 0.09809028058634237),
 ('letters', 0.06320196234999097),
 ('wrote', 0.061174121743173854),
 ('newspapers', 0.05901876343478673),
 ('studied', 0.05901876343478673),
 ('reading', 0.0557890219264284),
 ('chains', 0.046083653670617904)]

<h4 style="color:red">2b - Frank Drummer</h4>

In [37]:
topic_model.find_topics("""Frank Drummer Out of a cell into this darkened space— The end at twenty-five! My tongue could not speak what stirred within me, And the village thought me a fool. Yet at the start there was a clear vision, A high and urgent purpose in my soul Which drove me on trying to memorize The Encyclopedia Britannica!""")

([48, 20, 65, 2, 66],
 [0.32912258547430884,
  0.29779427169339073,
  0.2931649312090999,
  0.29126423135717056,
  0.27591233261544335])

In [38]:
topic_model.get_topic(48)

[('memory', 0.2528549455114603),
 ('memories', 0.19701242270268846),
 ('forget', 0.08909506376319462),
 ('page', 0.08146030702887726),
 ('stand', 0.07959693393903268),
 ('forgotten', 0.06852023353501803),
 ('remember', 0.06321373637786508),
 ('shines', 0.05110453347639859),
 ('urns', 0.05110453347639859),
 ('rupturing', 0.05110453347639859)]

In [39]:
topic_model.get_topic(20)

[('voice', 0.09473506868566467),
 ('song', 0.08524436636465511),
 ('music', 0.0816994643909253),
 ('singing', 0.06535957151274026),
 ('sound', 0.06291414521019748),
 ('trumpets', 0.060805679417289885),
 ('rumbled', 0.04433640613808263),
 ('strings', 0.04433640613808263),
 ('mingled', 0.04433640613808263),
 ('waves', 0.0405371196115266)]

In [40]:
topic_model.get_topic(65)

[('dreams', 0.4298658477876101),
 ('dream', 0.16602979664254366),
 ('dreamed', 0.14392279530977592),
 ('pageant', 0.08255347715418233),
 ('abruptly', 0.08255347715418233),
 ('mirrored', 0.08255347715418233),
 ('glassed', 0.08255347715418233),
 ('slept', 0.08255347715418233),
 ('borne', 0.08255347715418233),
 ('leafy', 0.07196139765488796)]

In [41]:
topic_model.get_topic(2)

[('ernest', 0.05872945577089072),
 ('hyde', 0.04716911160679473),
 ('tom', 0.04716911160679473),
 ('roy', 0.04716911160679473),
 ('hamilton', 0.04716911160679473),
 ('john', 0.046912609423265236),
 ('willie', 0.04163665974656238),
 ('george', 0.036603542204094675),
 ('enoch', 0.034393315055644975),
 ('dunlap', 0.034393315055644975)]

In [42]:
topic_model.get_topic(66)

[('darkness', 0.31342885429973166),
 ('light', 0.16073044667956157),
 ('bellowing', 0.12959594300048652),
 ('dark', 0.10900946244207413),
 ('fantastic', 0.08130266689427047),
 ('leading', 0.08130266689427047),
 ('plate', 0.08130266689427047),
 ('quenches', 0.08130266689427047),
 ('skipping', 0.08130266689427047),
 ('lantern', 0.08130266689427047)]

<h4 style="color:red">3a - Un giudice</h4>

In [43]:
topic_model.find_topics("""A judge What does it mean to be five foot tall, it’s revealed in the eyes and the comments of people, or the curiosity of an irreverent girl who approaches only on account of her impertinent doubt: she wants to discover whether it’s true what they say about dwarfs, that they’re better endowed with the least apparent virtue which, of all the virtues, is the most indecent. Years pass, as do months, and, if you count them, so do minutes, it’s sad to find yourself an adult without having grown up; backbiting insists, it beats its tongue on a drum until it says that a dwarf is certainly a skunk because his heart is too close to his arsehole. It was in the sleepless nights watched over by the light of ill feeling that I studied for my exams, I became an attorney to take the road that from the pews of a cathedral led to the vestry then to the bench of a court, a judge at last, the arbiter on earth of good and bad. And so my height no longer bestowed good humour on he who, standing in the dock, called me “Your Honour”, and entrusting him to the executioner was entirely my pleasure, before genuflecting in the hour of farewell not knowing at all the height of God.""")

([7, 73, 63, 68, 81],
 [0.34104475773599574,
  0.3356424567543312,
  0.2555459585970836,
  0.2551527459439523,
  0.24919341246923343])

In [44]:
topic_model.get_topic(7)

[('court', 0.15678098002478263),
 ('judge', 0.1422452404022924),
 ('county', 0.10458982787210781),
 ('house', 0.06421800110020533),
 ('somers', 0.056633466891432026),
 ('circuit', 0.05452017737092927),
 ('jury', 0.04089013302819695),
 ('superintendent', 0.04089013302819695),
 ('attorney', 0.04089013302819695),
 ('state', 0.04089013302819695)]

In [45]:
topic_model.get_topic(73)

[('jesus', 0.17530766426934144),
 ('blasphemer', 0.1585590117819565),
 ('unto', 0.1585590117819565),
 ('confucius', 0.1585590117819565),
 ('christ', 0.12796803899508435),
 ('thou', 0.112499022367387),
 ('said', 0.10636596923788441),
 ('god', 0.10214359192199404),
 ('crop', 0.09094874601731952),
 ('shalt', 0.09094874601731952)]

In [46]:
topic_model.get_topic(63)

[('cursed', 0.37092317280697473),
 ('god', 0.24723987378041634),
 ('particular', 0.14392279530977592),
 ('curse', 0.12288009207447904),
 ('alienated', 0.08255347715418233),
 ('betrayed', 0.08255347715418233),
 ('cursing', 0.08255347715418233),
 ('damned', 0.08255347715418233),
 ('sex', 0.08255347715418233),
 ('gravitation', 0.07196139765488796)]

In [47]:
topic_model.get_topic(68)

[('wisdom', 0.5686322224635225),
 ('material', 0.1012448304721104),
 ('gifted', 0.1012448304721104),
 ('boasted', 0.1012448304721104),
 ('brotherhood', 0.1012448304721104),
 ('intelligence', 0.1012448304721104),
 ('pope', 0.1012448304721104),
 ('dreamers', 0.1012448304721104),
 ('serpent', 0.1012448304721104),
 ('thuribles', 0.1012448304721104)]

In [48]:
topic_model.get_topic(81)

[('hatred', 0.23460732134272475),
 ('admonitions', 0.11665165250047502),
 ('hooted', 0.11665165250047502),
 ('termagant', 0.11665165250047502),
 ('slew', 0.11665165250047502),
 ('bullying', 0.11665165250047502),
 ('intangible', 0.11665165250047502),
 ('yahoos', 0.11665165250047502),
 ('degradation', 0.11665165250047502),
 ('nemesis', 0.11665165250047502)]

<h4 style="color:red">3b - Judge Selah Lively</h4>

In [49]:
topic_model.find_topics("""Judge Selah Lively Suppose you stood just five feet two, And had worked your way as a grocery clerk, Studying law by candle light Until you became an attorney at law? And then suppose through your diligence, And regular church attendance, You became attorney for Thomas Rhodes, Collecting notes and mortgages, And representing all the widows In the Probate Court? And through it all They jeered at your size, and laughed at your clothes And your polished boots? And then suppose You became the County Judge? And Jefferson Howard and Kinsey Keene, And Harmon Whitney, and all the giants Who had sneered at you, were forced to stand Before the bar and say "Your Honor"— Well, don’t you think it was natural That I made it hard for them?""")

([7, 73, 30, 18, 81],
 [0.3146689623877266,
  0.19578106477257035,
  0.17839136350939439,
  0.17501414684833366,
  0.16618117744291616])

In [50]:
topic_model.get_topic(7)

[('court', 0.15678098002478263),
 ('judge', 0.1422452404022924),
 ('county', 0.10458982787210781),
 ('house', 0.06421800110020533),
 ('somers', 0.056633466891432026),
 ('circuit', 0.05452017737092927),
 ('jury', 0.04089013302819695),
 ('superintendent', 0.04089013302819695),
 ('attorney', 0.04089013302819695),
 ('state', 0.04089013302819695)]

In [51]:
topic_model.get_topic(73)

[('jesus', 0.17530766426934144),
 ('blasphemer', 0.1585590117819565),
 ('unto', 0.1585590117819565),
 ('confucius', 0.1585590117819565),
 ('christ', 0.12796803899508435),
 ('thou', 0.112499022367387),
 ('said', 0.10636596923788441),
 ('god', 0.10214359192199404),
 ('crop', 0.09094874601731952),
 ('shalt', 0.09094874601731952)]

In [52]:
topic_model.get_topic(30)

[('rhodes', 0.4468913299099764),
 ('thomas', 0.2513378589707059),
 ('slave', 0.0971969572503649),
 ('bank', 0.07733472583714027),
 ('old', 0.06757489881765152),
 ('editor', 0.047039192097970806),
 ('croquet', 0.040651333447135236),
 ('gingham', 0.040651333447135236),
 ('lloyd', 0.040651333447135236),
 ('driver', 0.040651333447135236)]

In [53]:
topic_model.get_topic(18)

[('church', 0.22971500885858617),
 ('congregation', 0.062281545422563915),
 ('bell', 0.0581592668799112),
 ('sunday', 0.05497656044206294),
 ('faith', 0.05238804262992883),
 ('christian', 0.048330890677251215),
 ('prayer', 0.045412532500657445),
 ('sermons', 0.045412532500657445),
 ('presbyterian', 0.041521030281709274),
 ('ruled', 0.041521030281709274)]

In [54]:
topic_model.get_topic(81)

[('hatred', 0.23460732134272475),
 ('admonitions', 0.11665165250047502),
 ('hooted', 0.11665165250047502),
 ('termagant', 0.11665165250047502),
 ('slew', 0.11665165250047502),
 ('bullying', 0.11665165250047502),
 ('intangible', 0.11665165250047502),
 ('yahoos', 0.11665165250047502),
 ('degradation', 0.11665165250047502),
 ('nemesis', 0.11665165250047502)]

<h4 style="color:red">4a - Un blasfemo</h4>

In [55]:
topic_model.find_topics("""A blasphemous I never bowed again not even over a flower, nor did I blush when stealing love from the moment that winter convinced me that God wouldn’t have blushed stealing mine. They arrested me one day for women and wine, they didn’t have any laws for punishing a blasphemer, it wasn’t death that killed me, but two bigoted guards, who sought my soul with the power of blows. Because I said that God deceived the first man, he obliged him to travel through life as an idiot, in the enchanted garden he forced him to dream, to ignore the fact that in the world there’s good and bad. When he saw that man stretched his fingers to steal the mystery of a forbidden apple for fear that he no longer had any masters he stopped him with death and invented the seasons. And if it was two guards who stopped my life, it’s right here on earth the forbidden apple, and not God, but someone who invented him for us, obliges us to dream in an enchanted garden, obliges us to dream in an enchanted garden.""")

([73, 63, 65, 61, 4],
 [0.3794195440627878,
  0.29734084135619954,
  0.2715857609867122,
  0.2655382977467361,
  0.26027193899061474])

In [56]:
topic_model.get_topic(73)

[('jesus', 0.17530766426934144),
 ('blasphemer', 0.1585590117819565),
 ('unto', 0.1585590117819565),
 ('confucius', 0.1585590117819565),
 ('christ', 0.12796803899508435),
 ('thou', 0.112499022367387),
 ('said', 0.10636596923788441),
 ('god', 0.10214359192199404),
 ('crop', 0.09094874601731952),
 ('shalt', 0.09094874601731952)]

In [57]:
topic_model.get_topic(63)

[('cursed', 0.37092317280697473),
 ('god', 0.24723987378041634),
 ('particular', 0.14392279530977592),
 ('curse', 0.12288009207447904),
 ('alienated', 0.08255347715418233),
 ('betrayed', 0.08255347715418233),
 ('cursing', 0.08255347715418233),
 ('damned', 0.08255347715418233),
 ('sex', 0.08255347715418233),
 ('gravitation', 0.07196139765488796)]

In [58]:
topic_model.get_topic(65)

[('dreams', 0.4298658477876101),
 ('dream', 0.16602979664254366),
 ('dreamed', 0.14392279530977592),
 ('pageant', 0.08255347715418233),
 ('abruptly', 0.08255347715418233),
 ('mirrored', 0.08255347715418233),
 ('glassed', 0.08255347715418233),
 ('slept', 0.08255347715418233),
 ('borne', 0.08255347715418233),
 ('leafy', 0.07196139765488796)]

In [59]:
topic_model.get_topic(61)

[('thief', 0.29167396707473886),
 ('stole', 0.1619028240170501),
 ('theft', 0.12641867155588424),
 ('steal', 0.11558557078421772),
 ('apple', 0.10793521601136673),
 ('stealing', 0.10202857163121593),
 ('proctor', 0.07251318939218718),
 ('husbands', 0.07251318939218718),
 ('regard', 0.07251318939218718),
 ('prestige', 0.07251318939218718)]

In [60]:
topic_model.get_topic(4)

[('soul', 0.21098824295264204),
 ('souls', 0.092059138048087),
 ('your', 0.05062477628038193),
 ('body', 0.037047573728712084),
 ('cured', 0.028593816174848942),
 ('snared', 0.022326925286719413),
 ('angel', 0.022326925286719413),
 ('beloved', 0.01645688495129856),
 ('miles', 0.01645688495129856),
 ('truth', 0.016396226568882)]

<h4 style="color:red">4b - Wendell P. Bloyd</h4>

In [64]:
topic_model.find_topics('''Wendell P. Bloyd They first charged me with disorderly conduct, There being no statute on blasphemy. Later they locked me up as insane Where I was beaten to death by a Catholic guard. My offense was this: I said God lied to Adam, and destined him to lead the life of a fool, Ignorant that there is evil in the world as well as good. And when Adam outwitted God by eating the apple And saw through the lie, God drove him out of Eden to keep him from taking The fruit of immortal life. For Christ's sake, you sensible people, Here's what God Himself says about it in the book of Genesis: "And the Lord God said, behold the man Is become as one of us" (a little envy, you see), "To know good and evil" (The all-is-good lie exposed): "And now lest he put forth his hand and take Also of the tree of life and eat, and live forever: Therefore the Lord God sent Him forth from the Garden of Eden." The reason I believe God crucified His Own Son To get out of the wretched tangle is, because it sounds just like Him.''')

([73, 63, 82, 81, 4],
 [0.26664065240983337,
  0.23380105960797132,
  0.1690176498108017,
  0.15354344394301347,
  0.15345244452955986])

In [65]:
topic_model.get_topic(73)

[('jesus', 0.17530766426934144),
 ('blasphemer', 0.1585590117819565),
 ('unto', 0.1585590117819565),
 ('confucius', 0.1585590117819565),
 ('christ', 0.12796803899508435),
 ('thou', 0.112499022367387),
 ('said', 0.10636596923788441),
 ('god', 0.10214359192199404),
 ('crop', 0.09094874601731952),
 ('shalt', 0.09094874601731952)]

In [66]:
topic_model.get_topic(63)

[('cursed', 0.37092317280697473),
 ('god', 0.24723987378041634),
 ('particular', 0.14392279530977592),
 ('curse', 0.12288009207447904),
 ('alienated', 0.08255347715418233),
 ('betrayed', 0.08255347715418233),
 ('cursing', 0.08255347715418233),
 ('damned', 0.08255347715418233),
 ('sex', 0.08255347715418233),
 ('gravitation', 0.07196139765488796)]

In [68]:
topic_model.get_topic(82)

[('tereus', 0.2958224438830051),
 ('wrath', 0.2664675748584034),
 ('procne', 0.15839504144503908),
 ('cabanis', 0.1332337874292017),
 ('avenger', 0.09936992620410835),
 ('pagan', 0.09936992620410835),
 ('valor', 0.09936992620410835),
 ('chronicles', 0.09936992620410835),
 ('carnivorous', 0.09936992620410835),
 ('believer', 0.09936992620410835)]

In [69]:
topic_model.get_topic(81)

[('hatred', 0.23460732134272475),
 ('admonitions', 0.11665165250047502),
 ('hooted', 0.11665165250047502),
 ('termagant', 0.11665165250047502),
 ('slew', 0.11665165250047502),
 ('bullying', 0.11665165250047502),
 ('intangible', 0.11665165250047502),
 ('yahoos', 0.11665165250047502),
 ('degradation', 0.11665165250047502),
 ('nemesis', 0.11665165250047502)]

In [70]:
topic_model.get_topic(4)

[('soul', 0.21098824295264204),
 ('souls', 0.092059138048087),
 ('your', 0.05062477628038193),
 ('body', 0.037047573728712084),
 ('cured', 0.028593816174848942),
 ('snared', 0.022326925286719413),
 ('angel', 0.022326925286719413),
 ('beloved', 0.01645688495129856),
 ('miles', 0.01645688495129856),
 ('truth', 0.016396226568882)]

<h4 style="color:red">5a - Un malato di cuore</h4>

In [71]:
topic_model.find_topics("""A heart patient "I began to dream with them then the soul suddenly flew away." As a boy spying the kids playing at the foolish rhythm of your sick heart and you feel the urge to go out and try what you miss to run on the lawn, and you keep your urge for you, and linger thinking how the hell do they regain their breath. As a man feeling the wasted time spent letting your eyes narrating the life to you, and never being able to drink from the cup in one gulp but in small intermittent sips, and never being able to drink from the cup in one gulp but in small intermittent sips. Yet I have donated a smile and it still returns in every summer of its own when I guided her, or perhaps I was guided to count her hair with sweaty hands. I don't think I asked promises to her eyes, I don't think I chose silence or voice, when the heart stunned and now no, I don't remember if it was too much shock or too much happiness, and my heart went crazy and now no, I don't remember, from which horizon the light started to blur. And in the mild magic of the grass between long caresses on the face, her pearl-colored thighs remained maybe an uncaught flower . But that I kissed her, this yes, I remember with my heart on the lips, but that kissed her, by God, yes I remember, and my heart remained on her lips. "And the soul suddenly flew away but I would not dream with them no you can not dream with them.""")

([42, 4, 54, 65, 40],
 [0.39710606794902226,
  0.34043799251497847,
  0.3348089893528298,
  0.3073504152502683,
  0.29594852496252605])

In [72]:
topic_model.get_topic(42)

[('heart', 0.3049919769890169),
 ('hearts', 0.13130633566452135),
 ('took', 0.05699483627837379),
 ('poor', 0.04732687700612575),
 ('slowed', 0.046660661000190014),
 ('wounds', 0.046660661000190014),
 ('cored', 0.046660661000190014),
 ('bullet', 0.046660661000190014),
 ('pulse', 0.046660661000190014),
 ('brimming', 0.046660661000190014)]

In [73]:
topic_model.get_topic(4)

[('soul', 0.21098824295264204),
 ('souls', 0.092059138048087),
 ('your', 0.05062477628038193),
 ('body', 0.037047573728712084),
 ('cured', 0.028593816174848942),
 ('snared', 0.022326925286719413),
 ('angel', 0.022326925286719413),
 ('beloved', 0.01645688495129856),
 ('miles', 0.01645688495129856),
 ('truth', 0.016396226568882)]

In [74]:
topic_model.get_topic(54)

[('spirit', 0.459948962202499),
 ('spirits', 0.2374409484841526),
 ('aflame', 0.06624661746940556),
 ('audacity', 0.06624661746940556),
 ('surely', 0.06624661746940556),
 ('swarmed', 0.06624661746940556),
 ('excess', 0.06624661746940556),
 ('girded', 0.06624661746940556),
 ('vanquished', 0.06624661746940556),
 ('through', 0.06570699460035699)]

In [75]:
topic_model.get_topic(65)

[('dreams', 0.4298658477876101),
 ('dream', 0.16602979664254366),
 ('dreamed', 0.14392279530977592),
 ('pageant', 0.08255347715418233),
 ('abruptly', 0.08255347715418233),
 ('mirrored', 0.08255347715418233),
 ('glassed', 0.08255347715418233),
 ('slept', 0.08255347715418233),
 ('borne', 0.08255347715418233),
 ('leafy', 0.07196139765488796)]

In [76]:
topic_model.get_topic(40)

[('tears', 0.27626048808019465),
 ('cries', 0.14988801085785192),
 ('crying', 0.11991040868628153),
 ('wept', 0.07795818079279529),
 ('cry', 0.06656004987367614),
 ('heard', 0.053424213408934336),
 ('cried', 0.05174311130776788),
 ('sitters', 0.04471646679184876),
 ('steer', 0.04471646679184876),
 ('yells', 0.04471646679184876)]

<h4 style="color:red">5b - Francis Turner</h4>

In [77]:
topic_model.find_topics("""Francis Turner I could not run or play in boyhood. In manhood I could only sip the cup, Not drink- For scarlet-fever left my heart diseased. Yet I lie here Soothed by a secret none but Mary knows: There is a garden of acacia, Catalpa trees, and arbors sweet with vines– There on that afternoon in June By Mary’s side– Kissing her with my soul upon my lips It suddenly took flight.""")

([24, 23, 5, 72, 44],
 [0.32913700714074146,
  0.29186976112118235,
  0.28598674461727175,
  0.2781177049736428,
  0.27726121609299237])

In [78]:
topic_model.get_topic(24)

[('bouquet', 0.1048626986209719),
 ('blossoms', 0.08874673316490153),
 ('clover', 0.08389015889677753),
 ('flowers', 0.08389015889677753),
 ('flower', 0.07127776865026758),
 ('weed', 0.06656004987367614),
 ('fragrant', 0.05197212052853019),
 ('petals', 0.05197212052853019),
 ('bloom', 0.05197212052853019),
 ('roses', 0.047518512433511725)]

In [79]:
topic_model.get_topic(23)

[('wine', 0.19970637692544196),
 ('drink', 0.10384690955714),
 ('rum', 0.07728914672920581),
 ('drinks', 0.07728914672920581),
 ('thirsting', 0.056355311416478523),
 ('drank', 0.056355311416478523),
 ('drinking', 0.05152609781947055),
 ('cup', 0.05152609781947055),
 ('bread', 0.04811569870386228),
 ('dying', 0.03861991330766338)]

In [80]:
topic_model.get_topic(5)

[('mrs', 0.12549066161559208),
 ('emily', 0.10871180982468169),
 ('daisy', 0.09722465569157962),
 ('fraser', 0.08969516648210586),
 ('sparks', 0.08502380969267992),
 ('williams', 0.07195681067424449),
 ('mary', 0.05979677765473724),
 ('mcneely', 0.046590775639149304),
 ('armstrong', 0.042139557185294746),
 ('hannah', 0.042139557185294746)]

In [81]:
topic_model.get_topic(72)

[('secret', 0.48172501843375043),
 ('cried', 0.10705471305055424),
 ('knows', 0.10705471305055424),
 ('withdrew', 0.09251682784520433),
 ('disease', 0.09251682784520433),
 ('sower', 0.09251682784520433),
 ('soothed', 0.09251682784520433),
 ('smiling', 0.09251682784520433),
 ('hides', 0.09251682784520433),
 ('learning', 0.09251682784520433)]

In [82]:
topic_model.get_topic(44)

[('sorrow', 0.37461224992669784),
 ('despair', 0.06926710367623833),
 ('hear', 0.051117297025066864),
 ('murmur', 0.04922913775249405),
 ('ovid', 0.04922913775249405),
 ('sorrows', 0.04922913775249405),
 ('answered', 0.04922913775249405),
 ('ours', 0.04922913775249405),
 ('misery', 0.04922913775249405),
 ('maiden', 0.04922913775249405)]

<h4 style="color:red">6a - Un medico</h4>

In [83]:
topic_model.find_topics('''A doctor As a child I wanted to cure the cherries when they were red I thought they were injured for me health had left them with the snow flowers they had lost. A dream, it was a dream but it wasn’t brief for this reason I swore I’d become a doctor and not for a God, nor even for a game: so that the cherries could bloom again, so that the cherries could bloom again. And when at long last I was a doctor I didn’t want to betray the child for the man and many came, they were called people cherries that were ill in every season. And colleagues who agreed, the colleagues who were happy in reading in my heart my wish to love sent me their best clients with the diagnosis on their face and it was the same for everyone: ill from hunger, unable to pay. And then I understood, I was obliged to understand, that being a doctor is just a trade that science isn’t something you can give to people if you don’t want to go down with the same illness, if you don’t want the system to take you for hunger. And the system surely will take you for hunger in your children, in your wife who now despises you, and so those snow flowers are closed in a bottle, the label said: the elixir of youth. And a judge, a judge with the face of a man sent me to shed sunsets in prison useless to me and to my fingers branded forever a swindler and a cheat doctor professor swindler cheat.''')

([44, 4, 63, 35, 68],
 [0.25053430233105256,
  0.24563216974101698,
  0.22617547026135515,
  0.20090975260428537,
  0.19531136710424413])

In [84]:
topic_model.get_topic(44)

[('sorrow', 0.37461224992669784),
 ('despair', 0.06926710367623833),
 ('hear', 0.051117297025066864),
 ('murmur', 0.04922913775249405),
 ('ovid', 0.04922913775249405),
 ('sorrows', 0.04922913775249405),
 ('answered', 0.04922913775249405),
 ('ours', 0.04922913775249405),
 ('misery', 0.04922913775249405),
 ('maiden', 0.04922913775249405)]

In [85]:
topic_model.get_topic(4)

[('soul', 0.21098824295264204),
 ('souls', 0.092059138048087),
 ('your', 0.05062477628038193),
 ('body', 0.037047573728712084),
 ('cured', 0.028593816174848942),
 ('snared', 0.022326925286719413),
 ('angel', 0.022326925286719413),
 ('beloved', 0.01645688495129856),
 ('miles', 0.01645688495129856),
 ('truth', 0.016396226568882)]

In [86]:
topic_model.get_topic(63)

[('cursed', 0.37092317280697473),
 ('god', 0.24723987378041634),
 ('particular', 0.14392279530977592),
 ('curse', 0.12288009207447904),
 ('alienated', 0.08255347715418233),
 ('betrayed', 0.08255347715418233),
 ('cursing', 0.08255347715418233),
 ('damned', 0.08255347715418233),
 ('sex', 0.08255347715418233),
 ('gravitation', 0.07196139765488796)]

In [87]:
topic_model.get_topic(35)

[('fever', 0.09869229505421667),
 ('worried', 0.09216006905585929),
 ('sick', 0.0901905769974283),
 ('aching', 0.07196139765488796),
 ('feverish', 0.07196139765488796),
 ('pain', 0.058077802313153674),
 ('deep', 0.05105724861289103),
 ('ribs', 0.04127673857709117),
 ('chested', 0.04127673857709117),
 ('cough', 0.04127673857709117)]

In [88]:
topic_model.get_topic(68)

[('wisdom', 0.5686322224635225),
 ('material', 0.1012448304721104),
 ('gifted', 0.1012448304721104),
 ('boasted', 0.1012448304721104),
 ('brotherhood', 0.1012448304721104),
 ('intelligence', 0.1012448304721104),
 ('pope', 0.1012448304721104),
 ('dreamers', 0.1012448304721104),
 ('serpent', 0.1012448304721104),
 ('thuribles', 0.1012448304721104)]

<h4 style="color:red">6b - Doctor Siegfried Iseman</h4>

In [90]:
topic_model.find_topics(''' Dr. Siegfried Iseman I said when they handed me my diploma, I said to myself I will be good And wise and brave and helpful to others; I said I will carry the Christian creed Into the practice of medicine! Somehow the world and the other doctors Know what's in your heart as soon as you make This high-souled resolution. And the way of it is they starve you out. And no one comes to you but the poor. And you find too late that being a doctor Is just a way of making a living. And when you are poor and have to carry The Christian creed and wife and children All on your back, it is too much! That's why I made the Elixir of Youth, Which landed me in the jail at Peoria Branded a swindler and a crook By the upright Federal Judge!''')

([18, 68, 73, 44, 7],
 [0.2877696755934911,
  0.21515142424468858,
  0.20850120352047807,
  0.1532385740914287,
  0.135067502220163])

In [91]:
topic_model.get_topic(18)

[('church', 0.22971500885858617),
 ('congregation', 0.062281545422563915),
 ('bell', 0.0581592668799112),
 ('sunday', 0.05497656044206294),
 ('faith', 0.05238804262992883),
 ('christian', 0.048330890677251215),
 ('prayer', 0.045412532500657445),
 ('sermons', 0.045412532500657445),
 ('presbyterian', 0.041521030281709274),
 ('ruled', 0.041521030281709274)]

In [92]:
topic_model.get_topic(68)

[('wisdom', 0.5686322224635225),
 ('material', 0.1012448304721104),
 ('gifted', 0.1012448304721104),
 ('boasted', 0.1012448304721104),
 ('brotherhood', 0.1012448304721104),
 ('intelligence', 0.1012448304721104),
 ('pope', 0.1012448304721104),
 ('dreamers', 0.1012448304721104),
 ('serpent', 0.1012448304721104),
 ('thuribles', 0.1012448304721104)]

In [93]:
topic_model.get_topic(73)

[('jesus', 0.17530766426934144),
 ('blasphemer', 0.1585590117819565),
 ('unto', 0.1585590117819565),
 ('confucius', 0.1585590117819565),
 ('christ', 0.12796803899508435),
 ('thou', 0.112499022367387),
 ('said', 0.10636596923788441),
 ('god', 0.10214359192199404),
 ('crop', 0.09094874601731952),
 ('shalt', 0.09094874601731952)]

In [94]:
topic_model.get_topic(44)

[('sorrow', 0.37461224992669784),
 ('despair', 0.06926710367623833),
 ('hear', 0.051117297025066864),
 ('murmur', 0.04922913775249405),
 ('ovid', 0.04922913775249405),
 ('sorrows', 0.04922913775249405),
 ('answered', 0.04922913775249405),
 ('ours', 0.04922913775249405),
 ('misery', 0.04922913775249405),
 ('maiden', 0.04922913775249405)]

In [95]:
topic_model.get_topic(7)

[('court', 0.15678098002478263),
 ('judge', 0.1422452404022924),
 ('county', 0.10458982787210781),
 ('house', 0.06421800110020533),
 ('somers', 0.056633466891432026),
 ('circuit', 0.05452017737092927),
 ('jury', 0.04089013302819695),
 ('superintendent', 0.04089013302819695),
 ('attorney', 0.04089013302819695),
 ('state', 0.04089013302819695)]

<h4 style="color:red">7a - Un chimico</h4>

In [96]:
topic_model.find_topics('''A chemist Only death took me up the hill: a corpse among the many that give phosphorous to the air for bivouacs of fi res that say will-of-the-wisp that leave no ashes, that don’t melt the frost. Only death took me up the hill. As a chemist one day I had the power to marry the elements and make them react, but I never managed to understand how men could combine through love, entrusting joy and pain to a game. Look at the smile, look at the colour how they play on the face of those who seek love: but the same smile, the same colour where are they on the face of those who’ve had love, where are they on the face of those who’ve had love. It’s strange to leave without suffering, without the face of a woman you have to remember. But maybe your dying is different you who leave yourselves to love who surrender to April. What is there that’s different in your dying. Spring doesn’t knock, it enters self-assured like smoke it penetrates every crack it has lips of meat, hair of corn such fear, such desire that it takes you by the hand. Such fear, such desire that it takes you far. But look at hydrogen keeping silent in the sea look at oxygen sleeping silently at his side: only a law that I can understand was able to marry them without making them explode only a law that I can understand. I was a chemist and, no, I didn’t want to marry. I didn’t know with whom and whom I would have generated. I died in an experiment that went wrong just like the idiots who die of love. And someone will say there’s a better way.''')

([4, 44, 56, 24, 49],
 [0.3328861376241531,
  0.2916833585587474,
  0.29047791009318,
  0.2893245384303019,
  0.2724787431404516])

In [97]:
topic_model.get_topic(4)

[('soul', 0.21098824295264204),
 ('souls', 0.092059138048087),
 ('your', 0.05062477628038193),
 ('body', 0.037047573728712084),
 ('cured', 0.028593816174848942),
 ('snared', 0.022326925286719413),
 ('angel', 0.022326925286719413),
 ('beloved', 0.01645688495129856),
 ('miles', 0.01645688495129856),
 ('truth', 0.016396226568882)]

In [98]:
topic_model.get_topic(44)

[('sorrow', 0.37461224992669784),
 ('despair', 0.06926710367623833),
 ('hear', 0.051117297025066864),
 ('murmur', 0.04922913775249405),
 ('ovid', 0.04922913775249405),
 ('sorrows', 0.04922913775249405),
 ('answered', 0.04922913775249405),
 ('ours', 0.04922913775249405),
 ('misery', 0.04922913775249405),
 ('maiden', 0.04922913775249405)]

In [99]:
topic_model.get_topic(56)

[('love', 0.3983435410711366),
 ('soiling', 0.07350652075372398),
 ('disillusionment', 0.07350652075372398),
 ('improvisation', 0.07350652075372398),
 ('softened', 0.07350652075372398),
 ('thirsted', 0.07350652075372398),
 ('thwarted', 0.07350652075372398),
 ('feelings', 0.07350652075372398),
 ('passions', 0.07350652075372398),
 ('craved', 0.07350652075372398)]

In [100]:
topic_model.get_topic(24)

[('bouquet', 0.1048626986209719),
 ('blossoms', 0.08874673316490153),
 ('clover', 0.08389015889677753),
 ('flowers', 0.08389015889677753),
 ('flower', 0.07127776865026758),
 ('weed', 0.06656004987367614),
 ('fragrant', 0.05197212052853019),
 ('petals', 0.05197212052853019),
 ('bloom', 0.05197212052853019),
 ('roses', 0.047518512433511725)]

In [101]:
topic_model.get_topic(49)

[('fire', 0.25587820012639023),
 ('flame', 0.1738702372532694),
 ('burned', 0.1037686229015898),
 ('anarch', 0.05159592322136396),
 ('burnt', 0.05159592322136396),
 ('sovereign', 0.05159592322136396),
 ('crooned', 0.05159592322136396),
 ('devastating', 0.05159592322136396),
 ('bon', 0.05159592322136396),
 ('spurt', 0.05159592322136396)]

<h4 style="color:red">7b - Trainor the Druggist</h4>

In [102]:
topic_model.find_topics(''' Trainor the Druggist Only the chemist can tell, and not always the chemist, What will result from compounding Fluids or solids. And who can tell How men and women will interact On each other, or what children will result? There were Benjamin Pantier and his wife, Good in themselves, but evil toward each other: He oxygen, she hydrogen, Their son, a devastating fire. I Trainor, the druggist, a mixer of chemicals, Killed while making an experiment, Lived unwedded.''')

([79, 78, 9, 34, 51],
 [0.2684017378494531,
  0.23899681217119234,
  0.23422218882289697,
  0.2323545881506211,
  0.22826655685677533])

In [103]:
topic_model.get_topic(79)

[('pantier', 0.5970950236403818),
 ('benjamin', 0.4748818969683052),
 ('reuben', 0.17324040176176728),
 ('ben', 0.15839504144503908),
 ('variations', 0.09936992620410835),
 ('worthless', 0.09936992620410835),
 ('determinants', 0.09936992620410835),
 ('son', 0.08772381706614234),
 ('headed', 0.08662020088088364),
 ('threw', 0.08662020088088364)]

In [104]:
topic_model.get_topic(78)

[('blood', 0.7964767105908968),
 ('fountain', 0.13087746378102078),
 ('purer', 0.13087746378102078),
 ('champion', 0.13087746378102078),
 ('commissioned', 0.13087746378102078),
 ('vanquish', 0.13087746378102078),
 ('trash', 0.13087746378102078),
 ('honorable', 0.11408514262360286),
 ('blooded', 0.11408514262360286),
 ('filled', 0.11408514262360286)]

In [105]:
topic_model.get_topic(9)

[('married', 0.1231391183391663),
 ('husband', 0.09895619737644475),
 ('wife', 0.09393130574467423),
 ('marriage', 0.06863740273372707),
 ('divorce', 0.06863740273372707),
 ('years', 0.03732074245757388),
 ('affairs', 0.034018115255037944),
 ('marital', 0.034018115255037944),
 ('unwedded', 0.034018115255037944),
 ('we', 0.03277665234622944)]

In [106]:
topic_model.get_topic(34)

[('father', 0.2854358214259432),
 ('mother', 0.1826362634571423),
 ('malloy', 0.121018272497593),
 ('fathers', 0.07087107344799572),
 ('inherited', 0.054504731221037064),
 ('my', 0.05089034284627892),
 ('fortune', 0.047039192097970806),
 ('menard', 0.040651333447135236),
 ('countries', 0.040651333447135236),
 ('effect', 0.040651333447135236)]

In [107]:
topic_model.get_topic(51)

[('men', 0.2790397585779288),
 ('women', 0.1953356986425619),
 ('woman', 0.11161104989047584),
 ('other', 0.09452022046761588),
 ('neither', 0.05581019961216054),
 ('are', 0.055084105889217104),
 ('white', 0.051590605330854515),
 ('pioneer', 0.049684963102054176),
 ('tastes', 0.049684963102054176),
 ('endowed', 0.049684963102054176)]

<h4 style="color:red">8a - Un ottico</h4>

In [108]:
topic_model.find_topics('''An optician Colour-blind long-sighted, beggars of sight the merchant of light, your optician, now only wants special clients who don’t know what to do with normal eyes. No longer an optician, but a lens dealer for improvising happy eyes, because the pupils who are used to copying invent worlds on which they can watch. Follow with me these dreaming eyes, fleeing from the orbit and not wanting to return. “I see that I climb to steal the sun so as to have no more nights, I closed it in my eyes so as not to let it fall in nets of sunsets, and who will be cold will be cold along my glance they’ll need to warm up.” “I see the rivers inside my veins, they seek, they seek, they seek, they seek, they seek their sea, they break the banks, the banks, the banks, the banks, they fi nd skies, skies, skies, skies to photograph. Blood that runs without imagination brings tumours of melancholy.” “I see gendarmes graze women bent over the dew, red the tongues from the pollen of flowers but where’s the queen bee? Maybe she’s flown to dawn’s nests, maybe she’s flown, maybe she no longer flies.” “I see friends still on the road, they aren’t in a hurry, they still steal happiness from sleep and a bit of night from dawn, and then the light, light that transforms the world into a toy.” We’ll make glasses this way! We’ll make glasses this way!''')

([1, 65, 32, 66, 44],
 [0.3811272125760501,
  0.37399110064306146,
  0.355155688052623,
  0.35270439642193113,
  0.29716033892082105])

In [109]:
topic_model.get_topic(1)

[('eyes', 0.09959869363980786),
 ('eye', 0.08868976695859787),
 ('vision', 0.05336978227924318),
 ('face', 0.05170006236087284),
 ('smiles', 0.03419498346566964),
 ('brow', 0.02870765893806075),
 ('faces', 0.02735598677253571),
 ('look', 0.02378610639258431),
 ('laugh', 0.022777203378824537),
 ('beauty', 0.021530744203545563)]

In [110]:
topic_model.get_topic(65)

[('dreams', 0.4298658477876101),
 ('dream', 0.16602979664254366),
 ('dreamed', 0.14392279530977592),
 ('pageant', 0.08255347715418233),
 ('abruptly', 0.08255347715418233),
 ('mirrored', 0.08255347715418233),
 ('glassed', 0.08255347715418233),
 ('slept', 0.08255347715418233),
 ('borne', 0.08255347715418233),
 ('leafy', 0.07196139765488796)]

In [111]:
topic_model.get_topic(32)

[('sun', 0.30289139126360226),
 ('sky', 0.11705603566992213),
 ('light', 0.11512785483094176),
 ('stars', 0.07219969019688542),
 ('sunlight', 0.061916325463884786),
 ('cold', 0.048133126797923614),
 ('oxygen', 0.04159671329474303),
 ('hasty', 0.04159671329474303),
 ('whippoorwill', 0.04159671329474303),
 ('dip', 0.04159671329474303)]

In [112]:
topic_model.get_topic(66)

[('darkness', 0.31342885429973166),
 ('light', 0.16073044667956157),
 ('bellowing', 0.12959594300048652),
 ('dark', 0.10900946244207413),
 ('fantastic', 0.08130266689427047),
 ('leading', 0.08130266689427047),
 ('plate', 0.08130266689427047),
 ('quenches', 0.08130266689427047),
 ('skipping', 0.08130266689427047),
 ('lantern', 0.08130266689427047)]

In [113]:
topic_model.get_topic(44)

[('sorrow', 0.37461224992669784),
 ('despair', 0.06926710367623833),
 ('hear', 0.051117297025066864),
 ('murmur', 0.04922913775249405),
 ('ovid', 0.04922913775249405),
 ('sorrows', 0.04922913775249405),
 ('answered', 0.04922913775249405),
 ('ours', 0.04922913775249405),
 ('misery', 0.04922913775249405),
 ('maiden', 0.04922913775249405)]

<h4 style="color:red">8b - Dippold the Optician</h4>

In [114]:
topic_model.find_topics('''Dippold the Optician What do you see now? Globes of red, yellow, purple. Just a moment! And now? My father and mother and sisters. Yes! And now? Knights at arms, beautiful women, kind faces. Try this. A field of grain—a city. Very good! And now? A young woman with angels bending over her. A heavier lens! And now? Many women with bright eyes and open lips. Try this. Just a goblet on a table. Oh I see! Try this lens! Just an open space—I see nothing in particular. Well, now! Pine trees, a lake, a summer sky. That’s better. And now? A book. Read a page for me. I can’t. My eyes are carried beyond the page. Try this lens. Depths of air. Excellent! And now? Light, just light, making everything below it a toy world. Very well, we’ll make the glasses accordingly.''')

([32, 1, 65, 87, 66],
 [0.34706318481041676,
  0.33640667587533346,
  0.2949498005196969,
  0.28739028383579784,
  0.28105400072377834])

In [115]:
topic_model.get_topic(32)

[('sun', 0.30289139126360226),
 ('sky', 0.11705603566992213),
 ('light', 0.11512785483094176),
 ('stars', 0.07219969019688542),
 ('sunlight', 0.061916325463884786),
 ('cold', 0.048133126797923614),
 ('oxygen', 0.04159671329474303),
 ('hasty', 0.04159671329474303),
 ('whippoorwill', 0.04159671329474303),
 ('dip', 0.04159671329474303)]

In [116]:
topic_model.get_topic(1)

[('eyes', 0.09959869363980786),
 ('eye', 0.08868976695859787),
 ('vision', 0.05336978227924318),
 ('face', 0.05170006236087284),
 ('smiles', 0.03419498346566964),
 ('brow', 0.02870765893806075),
 ('faces', 0.02735598677253571),
 ('look', 0.02378610639258431),
 ('laugh', 0.022777203378824537),
 ('beauty', 0.021530744203545563)]

In [117]:
topic_model.get_topic(65)

[('dreams', 0.4298658477876101),
 ('dream', 0.16602979664254366),
 ('dreamed', 0.14392279530977592),
 ('pageant', 0.08255347715418233),
 ('abruptly', 0.08255347715418233),
 ('mirrored', 0.08255347715418233),
 ('glassed', 0.08255347715418233),
 ('slept', 0.08255347715418233),
 ('borne', 0.08255347715418233),
 ('leafy', 0.07196139765488796)]

In [118]:
topic_model.get_topic(87)

[('moon', 0.3880733348082591),
 ('moonlight', 0.16705324455598988),
 ('shone', 0.09582100026824733),
 ('middle', 0.09582100026824733),
 ('memphis', 0.09582100026824733),
 ('bigger', 0.09582100026824733),
 ('necropolis', 0.09582100026824733),
 ('lute', 0.09582100026824733),
 ('shoulder', 0.09582100026824733),
 ('thin', 0.09582100026824733)]

In [119]:
topic_model.get_topic(66)

[('darkness', 0.31342885429973166),
 ('light', 0.16073044667956157),
 ('bellowing', 0.12959594300048652),
 ('dark', 0.10900946244207413),
 ('fantastic', 0.08130266689427047),
 ('leading', 0.08130266689427047),
 ('plate', 0.08130266689427047),
 ('quenches', 0.08130266689427047),
 ('skipping', 0.08130266689427047),
 ('lantern', 0.08130266689427047)]

<h4 style="color:red">9a - Il suonatore Jones</h4>

In [120]:
topic_model.find_topics(''' Jones, the player In a whirlpool of dust the others saw drought it reminded me of Jenny’s skirt in a dance many years ago. I felt my earth vibrate with sounds, it was my heart, and so why till it again how to think it better. Liberty I have seen it sleeping in the tilled fields from heaven and money, from heaven and love, protected by barbed wire. Liberty I have seen it wake up every time I have played for a rustle of girls at a dance for a drunk companion. And then if people know and people know you can play, playing is up to you all life long and you like being listened to. It ended up with fields of weeds it ended up with a broken flute and many memories and a raucous laughter and not even one regret.''')

([83, 20, 65, 44, 24],
 [0.3635730159500624,
  0.24930244319910866,
  0.24564404492355282,
  0.2075862201962148,
  0.19755747332546753])

In [121]:
topic_model.get_topic(83)

[('dance', 0.3775057150354989),
 ('danced', 0.23961617954523412),
 ('rigadoon', 0.10731952030043702),
 ('rink', 0.10731952030043702),
 ('viols', 0.10731952030043702),
 ('picnic', 0.10731952030043702),
 ('chandlerville', 0.10731952030043702),
 ('merrily', 0.10731952030043702),
 ('dances', 0.10731952030043702),
 ('revival', 0.10731952030043702)]

In [122]:
topic_model.get_topic(20)

[('voice', 0.09473506868566467),
 ('song', 0.08524436636465511),
 ('music', 0.0816994643909253),
 ('singing', 0.06535957151274026),
 ('sound', 0.06291414521019748),
 ('trumpets', 0.060805679417289885),
 ('rumbled', 0.04433640613808263),
 ('strings', 0.04433640613808263),
 ('mingled', 0.04433640613808263),
 ('waves', 0.0405371196115266)]

In [123]:
topic_model.get_topic(65)

[('dreams', 0.4298658477876101),
 ('dream', 0.16602979664254366),
 ('dreamed', 0.14392279530977592),
 ('pageant', 0.08255347715418233),
 ('abruptly', 0.08255347715418233),
 ('mirrored', 0.08255347715418233),
 ('glassed', 0.08255347715418233),
 ('slept', 0.08255347715418233),
 ('borne', 0.08255347715418233),
 ('leafy', 0.07196139765488796)]

In [124]:
topic_model.get_topic(44)

[('sorrow', 0.37461224992669784),
 ('despair', 0.06926710367623833),
 ('hear', 0.051117297025066864),
 ('murmur', 0.04922913775249405),
 ('ovid', 0.04922913775249405),
 ('sorrows', 0.04922913775249405),
 ('answered', 0.04922913775249405),
 ('ours', 0.04922913775249405),
 ('misery', 0.04922913775249405),
 ('maiden', 0.04922913775249405)]

In [125]:
topic_model.get_topic(24)

[('bouquet', 0.1048626986209719),
 ('blossoms', 0.08874673316490153),
 ('clover', 0.08389015889677753),
 ('flowers', 0.08389015889677753),
 ('flower', 0.07127776865026758),
 ('weed', 0.06656004987367614),
 ('fragrant', 0.05197212052853019),
 ('petals', 0.05197212052853019),
 ('bloom', 0.05197212052853019),
 ('roses', 0.047518512433511725)]

<h4 style="color:red">9b - Jones the player</h4>

In [126]:
topic_model.find_topics(''' Fiddler Jones The earth keeps some vibration going There in your heart, and that is you. And if the people find you can fiddle, Why, fiddle you must, for all your life. What do you see, a harvest of clover? Or a meadow to walk through to the river? The wind's in the corn; you rub your hands For beeves hereafter ready for market; Or else you hear the rustle of skirts Like the girls when dancing at Little Grove. To Cooney Potter a pillar of dust Or whirling leaves meant ruinous drouth; They looked to me like Red-Head Sammy Stepping it off to 'Toor-a-Loor.' How could I till my forty acres Not to speak of getting more, With a medley of horns, bassoons and piccolos Stirred in my brain by crows and robins And the creak of a wind-mill--only these? And I never started to plow in my life That some one did not stop in the road And take me away to a dance or picnic. I ended up with forty acres; I ended up with a broken fiddle-- And a broken laugh, and a thousand memories, And not a single regret.''')

([6, 83, 60, 20, 29],
 [0.2753682595363104,
  0.2687721824848367,
  0.2563114568965506,
  0.2463868227356948,
  0.23075811882702835])

In [127]:
topic_model.get_topic(6)

[('spoon', 0.27083403589333854),
 ('river', 0.24581821278314295),
 ('spooniad', 0.026652369501810356),
 ('ship', 0.024368467914621397),
 ('train', 0.02151029715301988),
 ('all', 0.0210217683806952),
 ('in', 0.01642800045350359),
 ('born', 0.0158740324094937),
 ('brought', 0.0158740324094937),
 ('ditches', 0.01528768095447821)]

In [128]:
topic_model.get_topic(83)

[('dance', 0.3775057150354989),
 ('danced', 0.23961617954523412),
 ('rigadoon', 0.10731952030043702),
 ('rink', 0.10731952030043702),
 ('viols', 0.10731952030043702),
 ('picnic', 0.10731952030043702),
 ('chandlerville', 0.10731952030043702),
 ('merrily', 0.10731952030043702),
 ('dances', 0.10731952030043702),
 ('revival', 0.10731952030043702)]

In [129]:
topic_model.get_topic(60)

[('wind', 0.24225153807531688),
 ('winds', 0.1859420051746111),
 ('kite', 0.17363491271393777),
 ('blew', 0.12396133678307407),
 ('breeze', 0.11575660847595852),
 ('leaves', 0.11325504195073824),
 ('kites', 0.07776776833365001),
 ('blown', 0.07776776833365001),
 ('clocks', 0.07776776833365001),
 ('bucky', 0.07776776833365001)]

In [130]:
topic_model.get_topic(20)

[('voice', 0.09473506868566467),
 ('song', 0.08524436636465511),
 ('music', 0.0816994643909253),
 ('singing', 0.06535957151274026),
 ('sound', 0.06291414521019748),
 ('trumpets', 0.060805679417289885),
 ('rumbled', 0.04433640613808263),
 ('strings', 0.04433640613808263),
 ('mingled', 0.04433640613808263),
 ('waves', 0.0405371196115266)]

In [131]:
topic_model.get_topic(29)

[('wings', 0.12758575390956464),
 ('winged', 0.10941378061426216),
 ('hawk', 0.08206033546069662),
 ('chicken', 0.06407521708996872),
 ('eagles', 0.06407521708996872),
 ('crow', 0.06407521708996872),
 ('quail', 0.06407521708996872),
 ('feathers', 0.06407521708996872),
 ('brood', 0.05858446738378158),
 ('sunlight', 0.05470689030713108)]

<h1 style="color:red;">Keywords' extraction with KeyBERT</h1>

<p>After having found the most frequent topics for each song and for each poem, we extracted through the use of KeyBERT the keywords from the songs of the music album "Non al denaro non all'amore né al cielo" and from the poems of the SpoonRiver Anthology that inspired them. </p>
<p><b><a href="https://maartengr.github.io/KeyBERT/index.html">KeyBERT</a></b> is a keyword extraction library that leverages BERT embeddings to get keywords that are most representative of the underlying text document. We can say that it’s an unsupervised extractive way of getting keywords from a given text.</p> 

In [2]:
from keybert import KeyBERT

<p>We used as model <code>'all-mpnet-base-v2'</code>, that is, a sentence-transformers model. It maps sentences and paragraphs to a 768 dimensional dense vector space and can be used for tasks like clustering or semantic search.</p>

In [3]:
kw_model = KeyBERT('all-mpnet-base-v2')

 <p>In the variable <code>poems_and_songs</code> we stored a list containing all the songs of the music album and the poems from the Spoon River Anthology that inspired De André.</p>

In [4]:
poems_and_songs= ['''The Hill

Where are Elmer, Herman, Bert, Tom and Charley, The weak of will, the strong of arm, the clown, the boozer, the fighter? All, all are sleeping on the hill.

One passed in a fever,
One was burned in a mine,
One was killed in a brawl,
One died in a jail,
One fell from a bridge toiling for children and wife- All, all are sleeping, sleeping, sleeping on the hill.

Where are Ella, Kate, Mag, Lizzie and Edith, The tender heart, the simple soul, the loud, the proud, the happy one?– All, all are sleeping on the hill.

One died in shameful child-birth,
One of a thwarted love,
One at the hands of a brute in a brothel, One of a broken pride, in the search for heart’s desire; One after life in far-away London and Paris Was brought to her little space by Ella and Kate and Mag– All, all are sleeping, sleeping, sleeping on the hill.

Where are Uncle Isaac and Aunt Emily, And old Towny Kincaid and Sevigne Houghton, And Major Walker who had talked
With venerable men of the revolution?– All, all are sleeping on the hill.

They brought them dead sons from the war, And daughters whom life had crushed,
And their children fatherless, crying– All, all are sleeping, sleeping, sleeping on the hill. Where is Old Fiddler Jones
Who played with life all his ninety years, Braving the sleet with bared breast,
Drinking, rioting, thinking neither of wife nor kin, Nor gold, nor love, nor heaven?
Lo! he babbles of the fish-frys of long ago, Of the horse-races of long ago at Clary’s Grove, Of what Abe Lincoln said
One time at Springfield.''', 

'''
The hill
Where did Elmer go,
who let himself die of fever,
where is Herman, who burnt in a mine.
Where are Bert and Tom,
the first was killed in a fight
and the other came out of prison already dead.
And what will become of Charlie
who fell as he was working
and who flew from the bridge, flew onto the
street.
They’re sleeping, they’re sleeping on the hill
they’re sleeping, they’re sleeping on the hill.
Where are Ella and Kate
both dead because of a mistake
one from abortion, the other from love.
And Maggie killed in a brothel
by the caresses of an animal
and Edith consumed by a strange illness.
And Lizzie who followed life a long way
and from England
was brought back in this palm of earth.
They’re sleeping, they’re sleeping on the hill
they’re sleeping, they’re sleeping on the hill.
Where are the generals
who decorated themselves in battle
with cemeteries of crosses on their chest,
where are the sons of war
who left for an ideal, for a fraud,
for a love affair that ended badly.
They have sent back home their remains in flags
tied tightly so they would seem whole.
They’re sleeping, they’re sleeping on the hill
they’re sleeping, they’re sleeping on the hill.
Where is Jones the player
who was surprised by his 90 years
and with life would have played again
he who offered his face to the wind,
his throat to wine and never a thought
not to money, not to love, nor to the heaven.
He, yes, it seems you can hear him
still mumbling filth eaten in the street at the
wrong time.
It seems you can still hear him
saying to the liquor merchant:
“You who sell it, what do you buy that is better?”''',
'''(...)''']


<p>The <code>keyphrase_ngram_range</code> controls the length of the resulting keywords/keyphrases. We set it equal to (1, 1) in order to receive just a single word as result. We have also highlighted all the retrieved keywords in the input text just by setting <code>highlight</code> equal to <code>True</code>.</p>

<h4 style="color:red">1a - La collina</h4>

In [65]:
keywords_1a = kw_model.extract_keywords(poems_and_songs[1], 

                                     keyphrase_ngram_range=(1,1), 

                                     stop_words='english', 

                                     highlight=True,

                                     top_n=10)

print( list(dict(keywords_1a).keys()))

The hill Where did Elmer go who let himself die of fever where is Herman who burnt in mine Where are Bert and Tom 
the first was killed in fight and the other came out of prison already dead And what will become of Charlie who 
fell as he was working and who flew from the bridge flew onto the street They re sleeping they re sleeping on the 
hill they re sleeping they re sleeping on the hill Where are Ella and Kate both dead because of mistake one from 
abortion the other from love And Maggie killed in brothel by the caresses of an animal and Edith consumed by 
strange illness And Lizzie who followed life long way and from England was brought back in this palm of earth They 
re sleeping they re sleeping on the hill they re sleeping they re sleeping on the hill Where are the generals who 
decorated themselves in battle with cemeteries of crosses on their chest where are the sons of war who left for an 
ideal for fraud for love affair that ended badly They have sent back home their remains in flags tied tightly so 
they would seem whole They re sleeping they re sleeping on the hill they re sleeping they re sleeping on the hill 
Where is Jones the player who was surprised by his 90 years and with life would have played again he who offered 
his face to the wind his throat to wine and never thought not to money not to love nor to the heaven He yes it 
seems you can hear him still mumbling filth eaten in the street at the wrong time It seems you can still hear him 
saying to the liquor merchant You who sell it what do you buy that is better

['hill', 'cemeteries', 'remains', 'edith', 'lizzie', 'heaven', 'dead', 'bert', 'home', 'kate']


<h4 style="color:red">1b - The hill</h4>

In [64]:
keywords_1b = kw_model.extract_keywords(poems_and_songs[0], 

                                     keyphrase_ngram_range=(1,1), 

                                     stop_words='english', 

                                     highlight=True,

                                     top_n=10)

print( list(dict(keywords_1b).keys()))

The Hill Where are Elmer Herman Bert Tom and Charley The weak of will the strong of arm the clown the boozer the 
fighter All all are sleeping on the hill One passed in fever One was burned in mine One was killed in brawl One 
died in jail One fell from bridge toiling for children and wife All all are sleeping sleeping sleeping on the hill 
Where are Ella Kate Mag Lizzie and Edith The tender heart the simple soul the loud the proud the happy one All all 
are sleeping on the hill One died in shameful child birth One of thwarted love One at the hands of brute in brothel
One of broken pride in the search for heart desire One after life in far away London and Paris Was brought to her 
little space by Ella and Kate and Mag All all are sleeping sleeping sleeping on the hill Where are Uncle Isaac and 
Aunt Emily And old Towny Kincaid and Sevigne Houghton And Major Walker who had talked With venerable men of the 
revolution All all are sleeping on the hill They brought them dead sons from the war And daughters whom life had 
crushed And their children fatherless crying All all are sleeping sleeping sleeping on the hill Where is Old 
Fiddler Jones Who played with life all his ninety years Braving the sleet with bared breast Drinking rioting 
thinking neither of wife nor kin Nor gold nor love nor heaven Lo he babbles of the fish frys of long ago Of the 
horse races of long ago at Clary Grove Of what Abe Lincoln said One time at Springfield

['hill', 'bert', 'grove', 'edith', 'abe', 'charley', 'lincoln', 'lizzie', 'elmer', 'dead']


<h4 style="color:red">2a - Un matto</h4>

In [69]:
keywords_2a = kw_model.extract_keywords(poems_and_songs[3], 

                                     keyphrase_ngram_range=(1,1), 

                                     stop_words='english', 

                                     highlight=True,

                                     top_n=10)

print( list(dict(keywords_2a).keys()))

madman behind every idiot there village Try and have world in your heart and you can express it in words and 
daylight divides the piazza between village that laughs at you the idiot who passes and not even the night leaves 
you alone the others dream about themselves and you dream about them And yes even you would go and seek the words 
that are certain to make you heard to amaze for half an hour all you need is history book and tried to learn the 
encyclopedia by heart and after mackerel mackintosh and maculate the others continued until reading mad And without
knowing to whom owed my life gave it back in mental hospital here on the hill sleep unwillingly and yet now there 
light in my thoughts here in the semi darkness now invent words but miss light the light of the sun My bones still 
give to life they still give it flowery grass But life has remained in the mute voices of those who lost the idiot 
and who cry for him on the hill of those who still whisper with the same irony pitiful death snatched him from 
madness

['madman', 'madness', 'encyclopedia', 'village', 'mackintosh', 'mental', 'darkness', 'idiot', 'maculate', 'book']


<h4 style="color:red">2b - Frank Drummer</h4>

In [66]:
keywords_2b = kw_model.extract_keywords(poems_and_songs[2], 

                                     keyphrase_ngram_range=(1,1), 

                                     stop_words='english', 

                                     highlight=True,

                                     top_n=10)

print( list(dict(keywords_2b).keys()))

Frank Drummer Out of cell into this darkened space The end at twenty five My tongue could not speak what stirred 
within me And the village thought me fool Yet at the start there was clear vision high and urgent purpose in my 
soul Which drove me on trying to memorize The Encyclopedia Britannica

['drummer', 'frank', 'encyclopedia', 'britannica', 'end', 'memorize', 'purpose', 'cell', 'fool', 'space']


<h4 style="color:red">3a - Un giudice</h4>

In [71]:
keywords_3a = kw_model.extract_keywords(poems_and_songs[5], 

                                     keyphrase_ngram_range=(1,1), 

                                     stop_words='english', 

                                     highlight=True,

                                     top_n=10)

print( list(dict(keywords_3a).keys()))

judge What does it mean to be five foot tall it revealed in the eyes and the comments of people or the curiosity of
an irreverent girl who approaches only on account of her impertinent doubt she wants to discover whether it true 
what they say about dwarfs that they re better endowed with the least apparent virtue which of all the virtues is 
the most indecent Years pass as do months and if you count them so do minutes it sad to find yourself an adult 
without having grown up backbiting insists it beats its tongue on drum until it says that dwarf is certainly skunk 
because his heart is too close to his arsehole It was in the sleepless nights watched over by the light of ill 
feeling that studied for my exams became an attorney to take the road that from the pews of cathedral led to the 
vestry then to the bench of court judge at last the arbiter on earth of good and bad And so my height no longer 
bestowed good humour on he who standing in the dock called me Your Honour and entrusting him to the executioner was
entirely my pleasure before genuflecting in the hour of farewell not knowing at all the height of God

['judge', 'dwarf', 'court', 'dwarfs', 'height', 'tall', 'attorney', 'arbiter', 'honour', 'executioner']


<h4 style="color:red">3b - Judge Selah Lively</h4>

In [70]:
keywords_3b = kw_model.extract_keywords(poems_and_songs[4], 

                                     keyphrase_ngram_range=(1,1), 

                                     stop_words='english', 

                                     highlight=True,

                                     top_n=10)

print( list(dict(keywords_3b).keys()))

Judge Selah Lively Suppose you stood just five feet two And had worked your way as grocery clerk Studying law by 
candle light Until you became an attorney at law And then suppose through your diligence And regular church 
attendance You became attorney for Thomas Rhodes Collecting notes and mortgages And representing all the widows In 
the Probate Court And through it all They jeered at your size and laughed at your clothes And your polished boots 
And then suppose You became the County Judge And Jefferson Howard and Kinsey Keene And Harmon Whitney and all the 
giants Who had sneered at you were forced to stand Before the bar and say Your Honor Well don you think it was 
natural That made it hard for them

['judge', 'court', 'law', 'attorney', 'lively', 'honor', 'clerk', 'jefferson', 'probate', 'diligence']


<h4 style="color:red">4a - Un blasfemo</h4>

In [73]:
keywords_4a = kw_model.extract_keywords(poems_and_songs[7], 

                                     keyphrase_ngram_range=(1,1), 

                                     stop_words='english', 

                                     highlight=True,

                                     top_n=10)

print( list(dict(keywords_4a).keys()))

blasphemer behind every blasphemer there an enchanted garden never bowed again not even over flower nor did blush 
when stealing love from the moment that winter convinced me that God wouldn have blushed stealing mine They 
arrested me one day for women and wine they didn have any laws for punishing blasphemer it wasn death that killed 
me but two bigoted guards who sought my soul with the power of blows Because said that God deceived the first man 
he obliged him to travel through life as an idiot in the enchanted garden he forced him to dream to ignore the fact
that in the world there good and bad When he saw that man stretched his fingers to steal the mystery of forbidden 
apple for fear that he no longer had any masters he stopped him with death and invented the seasons And if it was 
two guards who stopped my life it right here on earth the forbidden apple and not God but someone who invented him 
for us obliges us to dream in an enchanted garden obliges us to dream in an enchanted garden

['blasphemer', 'enchanted', 'garden', 'god', 'forbidden', 'flower', 'punishing', 'soul', 'death', 'obliges']


<h4 style="color:red">4b - Wendell P. Bloyd</h4>

In [72]:
keywords_4b = kw_model.extract_keywords(poems_and_songs[6], 

                                     keyphrase_ngram_range=(1,1), 

                                     stop_words='english', 

                                     highlight=True,

                                     top_n=10)

print( list(dict(keywords_4b).keys()))

Wendell Bloyd They first charged me with disorderly conduct There being no statute on blasphemy Later they locked 
me up as insane Where was beaten to death by Catholic guard My offense was this said God lied to Adam and destined 
him To lead the life of fool Ignorant that there is evil in the world as well as good And when Adam outwitted God 
by eating the apple And saw through the lie God drove him out of Eden to keep him from taking The fruit of immortal
life For Christ sake you sensible people Here what God Himself said about it in the book of Genesis And the Lord 
God said behold the man is become one of us little envy you see To know good and evil the all is good lie exposed 
And now lest he put forth his hand and take Also of the three of life and eat and live forever Therefore the Lord 
God sent Him forth from the garden of Eden The reason believe God crucified His Own Son To get out of the wretched 
tangle is because it sounds just like Him

['genesis', 'catholic', 'god', 'adam', 'crucified', 'blasphemy', 'evil', 'wretched', 'death', 'eden']


<h4 style="color:red">5a - Un malato di cuore</h4>

In [11]:
keywords_5a = kw_model.extract_keywords(poems_and_songs[9], 

                                     keyphrase_ngram_range=(1,1), 

                                     stop_words='english', 

                                     highlight=True,

                                     top_n=10)

print( list(dict(keywords_5a).keys()))

heart patient began to dream with them then the soul suddenly flew away As boy spying the kids playing at the 
foolish rhythm of your sick heart and you feel the urge to go out and try what you miss to run on the lawn and you 
keep your urge for you and linger thinking how the hell do they regain their breath As man feeling the wasted time 
spent letting your eyes narrating the life to you and never being able to drink from the cup in one gulp but in 
small intermittent sips and never being able to drink from the cup in one gulp but in small intermittent sips Yet 
have donated smile and it still returns in every summer of its own when guided her or perhaps was guided to count 
her hair with sweaty hands don think asked promises to her eyes don think chose silence or voice when the heart 
stunned and now no don remember if it was too much shock or too much happiness and my heart went crazy and now no 
don remember from which horizon the light started to blur And in the mild magic of the grass between long caresses 
on the face her pearl colored thighs remained maybe an uncaught flower But that kissed her this yes remember with 
my heart on the lips but that kissed her by God yes remember and my heart remained on her lips And the soul 
suddenly flew away but would not dream with them no you can not dream with them

['heart', 'soul', 'caresses', 'flower', 'feel', 'patient', 'narrating', 'breath', 'feeling', 'rhythm']


<h4 style="color:red">5b - Francis Turner</h4>

In [74]:
keywords_5b = kw_model.extract_keywords(poems_and_songs[8], 

                                     keyphrase_ngram_range=(1,1), 

                                     stop_words='english', 

                                     highlight=True,

                                     top_n=10)

print( list(dict(keywords_5b).keys()))

Francis Turner could not run or play In boyhood In manhood could only sip the cup Not drink For scarlet fever left 
my heart diseased Yet lie here Soothed by secret none but Mary knows There is garden of acacia Catalpa trees and 
arbors sweet with vines There on that afternoon in June By Mary side Kissing her with my soul upon my lips It 
suddenly took flight

['boyhood', 'turner', 'francis', 'mary', 'garden', 'fever', 'manhood', 'scarlet', 'soul', 'heart']


<h4 style="color:red">6a - Un dottore</h4>

In [77]:
keywords_6a = kw_model.extract_keywords(poems_and_songs[11], 

                                     keyphrase_ngram_range=(1,1), 

                                     stop_words='english', 

                                     highlight=True,

                                     top_n=10)

print( list(dict(keywords_6a).keys()))

Doctor As child wanted to cure the cherries when they were red thought they were injured for me health had left 
them with the snow flowers they had lost dream it was dream but it wasn brief for this reason swore become doctor 
and not for God nor even for game so that the cherries could bloom again so that the cherries could bloom again And
when at long last was doctor didn want to betray the child for the man and many came they were called people 
cherries that were ill in every season And colleagues who agreed the colleagues who were happy in reading in my 
heart my wish to love sent me their best clients with the diagnosis on their face and it was the same for everyone 
ill from hunger unable to pay And then understood was obliged to understand that being doctor is just trade that 
science isn something you can give to people if you don want to go down with the same illness if you don want the 
system to take you for hunger And the system surely will take you for hunger in your children in your wife who now 
despises you and so those snow flowers are closed in bottle the label said the elixir of youth And judge judge with
the face of man sent me to shed sunsets in prison useless to me and to my fingers branded forever swindler and 
cheat doctor professor swindler cheat

['doctor', 'flowers', 'cherries', 'illness', 'cure', 'diagnosis', 'professor', 'health', 'children', 'child']


<h4 style="color:red">6b - Doctor Siegfried Iseman</h4>

In [76]:
keywords_6b = kw_model.extract_keywords(poems_and_songs[10], 

                                     keyphrase_ngram_range=(1,1), 

                                     stop_words='english', 

                                     highlight=True,

                                     top_n=10)

print( list(dict(keywords_6b).keys()))

Dr Siegfried Iseman said when they handed me my diploma said to myself will be good And wise and brave and helpful 
to others said will carry the Christian creed Into the practice of medicine Somehow the world and the other doctors
Know what in your heart as soon as you make This high souled resolution And the way of it is they starve you out 
And no one comes to you but the poor And you find too late that being doctor Is just way of making living And when 
you are poor and have to carry The Christian creed and wife and children All on your back it is too much That why 
made the Elixir of Youth Which landed me in the jail at Peoria Branded swindler and crook By the upright Federal 
Judge

['doctor', 'dr', 'doctors', 'medicine', 'iseman', 'swindler', 'diploma', 'christian', 'youth', 'siegfried']


<h4 style="color:red">7a - Un chimico</h4>

In [79]:
keywords_7a = kw_model.extract_keywords(poems_and_songs[13], 

                                     keyphrase_ngram_range=(1,1), 

                                     stop_words='english', 

                                     highlight=True,

                                     top_n=10)

print( list(dict(keywords_7a).keys()))

Chemist Only death took me up the hill corpse among the many that give phosphorous to the air for bivouacs of fi 
res that say will of the wisp that leave no ashes that don melt the frost Only death took me up the hill As chemist
one day had the power to marry the elements and make them react but never managed to understand how men could 
combine through love entrusting joy and pain to game Look at the smile look at the colour how they play on the face
of those who seek love but the same smile the same colour where are they on the face of those who ve had love where
are they on the face of those who ve had love It strange to leave without suffering without the face of woman you 
have to remember But maybe your dying is different you who leave yourselves to love who surrender to April What is 
there that different in your dying Spring doesn knock it enters self assured like smoke it penetrates every crack 
it has lips of meat hair of corn such fear such desire that it takes you by the hand Such fear such desire that it 
takes you far But look at hydrogen keeping silent in the sea look at oxygen sleeping silently at his side only law 
that can understand was able to marry them without making them explode only law that can understand was chemist and
no didn want to marry didn know with whom and whom would have generated died in an experiment that went wrong just 
like the idiots who die of love And someone will say there better way

['chemist', 'death', 'oxygen', 'phosphorous', 'hydrogen', 'ashes', 'corpse', 'dying', 'die', 'suffering']


<h4 style="color:red">7b - Trainor the Druggist</h4>

In [78]:
keywords_7b = kw_model.extract_keywords(poems_and_songs[12], 

                                     keyphrase_ngram_range=(1,1), 

                                     stop_words='english', 

                                     highlight=True,

                                     top_n=10)

print( list(dict(keywords_7b).keys()))

Trainor the Druggist Only the chemist can tell and not always the chemist What will result from compounding Fluids 
or solids And who can tell How men and women will interact On each other or what children will result There were 
Benjamin Pantier and his wife Good in themselves but evil toward each other He oxygen she hydrogen Their son 
devastating fire Trainor the druggist mixer of chemicals Killed while making an experiment Lived unwedded

['chemist', 'druggist', 'chemicals', 'trainor', 'compounding', 'hydrogen', 'experiment', 'mixer', 'pantier', 'fluids']


<h4 style="color:red">8a - Un ottico</h4>

In [81]:
keywords_8a = kw_model.extract_keywords(poems_and_songs[15], 

                                     keyphrase_ngram_range=(1,1), 

                                     stop_words='english', 

                                     highlight=True,

                                     top_n=10)

print( list(dict(keywords_8a).keys()))

AN Optician Colour blind long sighted beggars of sight the merchant of light your optician now only wants special 
clients who don know what to do with normal eyes No longer an optician but lens dealer for improvising happy eyes 
because the pupils who are used to copying invent worlds on which they can watch Follow with me these dreaming eyes
fleeing from the orbit and not wanting to return see that climb to steal the sun so as to have no more nights 
closed it in my eyes so as not to let it fall in nets of sunsets and who will be cold will be cold along my glance 
they ll need to warm up see the rivers inside my veins they seek they seek they seek they seek they seek their sea 
they break the banks the banks the banks the banks they fi nd skies skies skies skies to photograph Blood that runs
without imagination brings tumours of melancholy see gendarmes graze women bent over the dew red the tongues from 
the pollen of flowers but where the queen bee Maybe she flown to dawn nests maybe she flown maybe she no longer 
flies see friends still on the road they aren in hurry they still steal happiness from sleep and bit of night from 
dawn and then the light light that transforms the world into toy We ll make glasses this way We ll make glasses 
this way

['optician', 'glasses', 'sight', 'eyes', 'sighted', 'lens', 'pupils', 'light', 'blind', 'glance']


<h4 style="color:red">8b - Dippold the Optician</h4>

In [80]:
keywords_8b = kw_model.extract_keywords(poems_and_songs[14], 

                                     keyphrase_ngram_range=(1,1), 

                                     stop_words='english', 

                                     highlight=True,

                                     top_n=10)

print( list(dict(keywords_8b).keys()))

Dippold the Optician What do you see now Globes of red yellow purple Just moment And now My father and mother and 
sisters Yes And now Knights at arms beautiful women kind faces Try this field of grain city Very good And now young
woman with angels bending over her heavier lens And now Many women with bright eyes and open lips Try this Just 
goblet on table Oh see Try this lens Just an open space see nothing in particular Well now Pine trees lake summer 
sky That better And now book Read page for me can My eyes are carried beyond the page Try this lens Depths of air 
Excellent And now Light just light making everything below it toy world Very well we ll make the glasses 
accordingly

['dippold', 'optician', 'glasses', 'lens', 'eyes', 'globes', 'light', 'bending', 'depths', 'goblet']


<h4 style="color:red">9a - Il suonatore Jones</h4>

In [83]:
keywords_9a = kw_model.extract_keywords(poems_and_songs[17], 

                                     keyphrase_ngram_range=(1,1), 

                                     stop_words='english', 

                                     highlight=True,

                                     top_n=10)

print( list(dict(keywords_9a).keys()))

Jones the player In whirlpool of dust the others saw drought it reminded me of Jenny skirt in dance many years ago 
felt my earth vibrate with sounds it was my heart and so why till it again how to think it better Liberty have seen
it sleeping in the tilled fields from heaven and money from heaven and love protected by barbed wire Liberty have 
seen it wake up every time have played for rustle of girls at dance for drunk companion And then if people know and
people know you can play playing is up to you all life long and you like being listened to 299 It ended up with 
fields of weeds it ended up with broken flute and many memories and raucous laughter and not even one regret

['jones', 'flute', 'play', 'playing', 'player', 'dust', 'played', 'drought', 'sounds', 'listened']


<h4 style="color:red">9b - Jones the player</h4>

In [82]:
keywords_9b = kw_model.extract_keywords(poems_and_songs[16], 

                                     keyphrase_ngram_range=(1,1), 

                                     stop_words='english', 

                                     highlight=True,

                                     top_n=10)

print( list(dict(keywords_9b).keys()))

Fiddler Jones The earth keeps some vibration going There in your heart and that is you And if the people find you 
can fiddle Why fiddle you must for all your life What do you see harvest of clover Or meadow to walk through to the
river The wind in the corn you rub your hands For beeves hereafter ready for market Or else you hear the rustle of 
skirts Like the girls when dancing at Little Grove To Cooney Potter pillar of dust Or whirling leaves meant ruinous
drouth They looked to me like Red Head Sammy Stepping it off to Toor Loor How could till my forty acres Not to 
speak of getting more With medley of horns bassoons and piccolos Stirred in my brain by crows and robins And the 
creak of wind mill only these And never started to plow in my life That some one did not stop in the road And take 
me away to dance or picnic ended up with forty acres ended up with broken fiddle And broken laugh and thousand 
memories And not single regret

['fiddler', 'fiddle', 'jones', 'whirling', 'potter', 'bassoons', 'meadow', 'acres', 'rustle', 'cooney']


<h4 style="color:red;">Find topics with the keywords obtained from KeyBERT</h4>

<p>After the keywords' extraction with KeyBERT, we returned to BERTopic and to our model <code>topic_model</code> already used for the topics' detection and extraction with the entire Spoon River Anthology.</p>
<p>In this case, we used as parameter of the method <code>find_topics</code> one single keyword and, in most cases, we considered as closest topic to our keyword the one judged as the most similar to the input word also by our model .</p>
<p>It is important to highlight the fact that we decided to do not pass as input of the method <code>find_topic</code> all the proper names, because, in this project, we considered as "keywords" just the ones that remind to a particular conceptual topic.</p>

<h4 style="color:red;">1a - La collina</h4>
<p>['hill', 'cemeteries', 'remains', 'edith', 'lizzie', 'heaven', 'dead', 'bert', 'home', 'kate']</p>

In [17]:
similar_topics, similarity=topic_model.find_topics("hill", top_n=5)
topic_model.get_topic(similar_topics[0])

[('hill', 0.09884535193548062),
 ('mountains', 0.08710678027999316),
 ('hills', 0.08186179262185232),
 ('mountain', 0.06533008520999486),
 ('meadow', 0.06171183780383318),
 ('golden', 0.05228674402441735),
 ('slopes', 0.05110467196999194),
 ('valleys', 0.05110467196999194),
 ('plow', 0.046678427519008824),
 ('lawn', 0.04355339013999658)]

In [18]:
similar_topics, similarity=topic_model.find_topics("cemeteries", top_n=5)
topic_model.get_topic(similar_topics[0])

[('grave', 0.15679348639741028),
 ('dead', 0.13750869138150082),
 ('died', 0.06139162958986042),
 ('alive', 0.053911493696976155),
 ('cemetery', 0.0489415112516854),
 ('graven', 0.0489415112516854),
 ('funeral', 0.0489415112516854),
 ('buried', 0.04470262106317776),
 ('bring', 0.035940995797984106),
 ('shame', 0.035940995797984106)]

In [19]:
similar_topics, similarity=topic_model.find_topics("remains", top_n=5)
topic_model.get_topic(similar_topics[0])

[('grave', 0.15679348639741028),
 ('dead', 0.13750869138150082),
 ('died', 0.06139162958986042),
 ('alive', 0.053911493696976155),
 ('cemetery', 0.0489415112516854),
 ('graven', 0.0489415112516854),
 ('funeral', 0.0489415112516854),
 ('buried', 0.04470262106317776),
 ('bring', 0.035940995797984106),
 ('shame', 0.035940995797984106)]

In [24]:
similar_topics, similarity=topic_model.find_topics("heaven", top_n=5)
topic_model.get_topic(similar_topics[0])

[('heaven', 0.20895250724394462),
 ('earth', 0.17017759722309309),
 ('kingdom', 0.10510884820452508),
 ('gold', 0.10180003938926556),
 ('sphere', 0.08448795380940598),
 ('worlds', 0.07446561761662537),
 ('bidding', 0.06166630417712361),
 ('realms', 0.06166630417712361),
 ('losers', 0.06166630417712361),
 ('silver', 0.05255442410226254)]

In [25]:
similar_topics, similarity=topic_model.find_topics("dead", top_n=5)
topic_model.get_topic(similar_topics[0])

[('grave', 0.15679348639741028),
 ('dead', 0.13750869138150082),
 ('died', 0.06139162958986042),
 ('alive', 0.053911493696976155),
 ('cemetery', 0.0489415112516854),
 ('graven', 0.0489415112516854),
 ('funeral', 0.0489415112516854),
 ('buried', 0.04470262106317776),
 ('bring', 0.035940995797984106),
 ('shame', 0.035940995797984106)]

In [26]:
similar_topics, similarity=topic_model.find_topics("home", top_n=5)
topic_model.get_topic(similar_topics[0])

[('room', 0.1405844735136448),
 ('house', 0.13794252438617602),
 ('alone', 0.09708978757505192),
 ('school', 0.036018068898650314),
 ('dingy', 0.033273185707081085),
 ('little', 0.032375148777747303),
 ('houses', 0.030391350291153228),
 ('rooms', 0.030391350291153228),
 ('enter', 0.030391350291153228),
 ('hour', 0.030277215870497767)]

<h4 style="color:red;">1b - The hill</h4>
<p>['hill', 'bert', 'grove', 'edith', 'abe', 'charley', 'lincoln', 'lizzie', 'elmer', 'dead']</p>

In [287]:
similar_topics, similarity=topic_model.find_topics("hill", top_n=5)
topic_model.get_topic(similar_topics[0])

[('hill', 0.09884535193548062),
 ('mountains', 0.08710678027999316),
 ('hills', 0.08186179262185232),
 ('mountain', 0.06533008520999486),
 ('meadow', 0.06171183780383318),
 ('golden', 0.05228674402441735),
 ('slopes', 0.05110467196999194),
 ('valleys', 0.05110467196999194),
 ('plow', 0.046678427519008824),
 ('lawn', 0.04355339013999658)]

In [16]:
similar_topics, similarity=topic_model.find_topics("dead", top_n=5)
topic_model.get_topic(similar_topics[0])

[('grave', 0.15679348639741028),
 ('dead', 0.13750869138150082),
 ('died', 0.06139162958986042),
 ('alive', 0.053911493696976155),
 ('cemetery', 0.0489415112516854),
 ('graven', 0.0489415112516854),
 ('funeral', 0.0489415112516854),
 ('buried', 0.04470262106317776),
 ('bring', 0.035940995797984106),
 ('shame', 0.035940995797984106)]

<h4 style="color:red;">2a - Un matto</h4>
<p>['madman', 'madness', 'encyclopedia', 'village', 'mackintosh', 'mental', 'darkness', 'idiot', 'maculate', 'book']</p>

In [34]:
similar_topics, similarity=topic_model.find_topics("madman", top_n=5)
topic_model.get_topic(similar_topics[0])

[('fever', 0.06433092929650201),
 ('lungs', 0.06433092929650201),
 ('breathe', 0.06433092929650201),
 ('worried', 0.06002408844167042),
 ('sick', 0.05849887036007819),
 ('lips', 0.056699708845146225),
 ('feverish', 0.04695403871354589),
 ('aching', 0.04695403871354589),
 ('kiss', 0.042887286197668004),
 ('sickness', 0.042887286197668004)]

In [38]:
similar_topics, similarity=topic_model.find_topics("madness", top_n=5)
topic_model.get_topic(similar_topics[0])

[('world', 0.26816723488008276),
 ('learn', 0.11562432033210678),
 ('reform', 0.10560994226175746),
 ('nothing', 0.09794648777059904),
 ('peace', 0.08491060257273746),
 ('swallows', 0.06641507473802234),
 ('savers', 0.06641507473802234),
 ('reaped', 0.06641507473802234),
 ('pathways', 0.06641507473802234),
 ('miracle', 0.06641507473802234)]

In [39]:
similar_topics, similarity=topic_model.find_topics("encyclopedia", top_n=5)
topic_model.get_topic(similar_topics[0])

[('write', 0.0932537483783199),
 ('read', 0.08752642616979708),
 ('letter', 0.07997783889252948),
 ('books', 0.07721131081461165),
 ('elected', 0.05512701828908658),
 ('legislature', 0.044311863886052086),
 ('letters', 0.044311863886052086),
 ('wrote', 0.042713303240251285),
 ('studied', 0.041345263716814934),
 ('newspapers', 0.041345263716814934)]

In [40]:
similar_topics, similarity=topic_model.find_topics("village", top_n=5)
topic_model.get_topic(similar_topics[0])

[('village', 0.3060900327012269),
 ('town', 0.06850890134457051),
 ('galilee', 0.047435618597787395),
 ('atheist', 0.047435618597787395),
 ('really', 0.03235521596040869),
 ('in', 0.028745276686860124),
 ('appearing', 0.027247210148932242),
 ('trustees', 0.027247210148932242),
 ('meetings', 0.027247210148932242),
 ('merriment', 0.027247210148932242)]

In [41]:
similar_topics, similarity=topic_model.find_topics("mental", top_n=5)
topic_model.get_topic(similar_topics[0])

[('soul', 0.21204952130363175),
 ('souls', 0.09655057373443228),
 ('your', 0.042519229082855665),
 ('body', 0.02936539597767276),
 ('cured', 0.02842486880531027),
 ('snared', 0.02223544621771284),
 ('miles', 0.01632896203321874),
 ('beloved', 0.01632896203321874),
 ('truth', 0.016171739198973606),
 ('makes', 0.01468269798883638)]

In [42]:
similar_topics, similarity=topic_model.find_topics("darkness", top_n=5)
topic_model.get_topic(similar_topics[0])

[('darkness', 0.3087658275372271),
 ('light', 0.15720620307085756),
 ('bellowing', 0.1280120512263727),
 ('spear', 0.08050312089457254),
 ('staggered', 0.08050312089457254),
 ('lantern', 0.08050312089457254),
 ('plate', 0.08050312089457254),
 ('fantastic', 0.08050312089457254),
 ('skipping', 0.08050312089457254),
 ('splinters', 0.08050312089457254)]

In [43]:
similar_topics, similarity=topic_model.find_topics("idiot", top_n=5)
topic_model.get_topic(similar_topics[0])

[('could', 0.16381289258149084),
 ('nonsense', 0.15948182114773346),
 ('talk', 0.11711807251412061),
 ('slightest', 0.0916069996386515),
 ('sidle', 0.0916069996386515),
 ('margin', 0.0916069996386515),
 ('doubtful', 0.0916069996386515),
 ('deal', 0.0916069996386515),
 ('damn', 0.0916069996386515),
 ('ai', 0.0916069996386515)]

In [44]:
similar_topics, similarity=topic_model.find_topics("book", top_n=5)
topic_model.get_topic(similar_topics[0])

[('write', 0.0932537483783199),
 ('read', 0.08752642616979708),
 ('letter', 0.07997783889252948),
 ('books', 0.07721131081461165),
 ('elected', 0.05512701828908658),
 ('legislature', 0.044311863886052086),
 ('letters', 0.044311863886052086),
 ('wrote', 0.042713303240251285),
 ('studied', 0.041345263716814934),
 ('newspapers', 0.041345263716814934)]

<p>The terms "mackerel", "mackintosh" and "maculate", of which the last two have been highlighted by KeyBERT as keywords of the song "Un matto", have been translated in this way from the italian "maiale", "Majakovskij", "malfatto" to keep the idea that the others read all the words in the encyclopedia till the word "matto", or "madman" in english, referred to the main character of the song.</br>
For this reason we decided to do not consider them as important for highlighting the main concepts of the song.</p>

<h4 style="color:red;">2b - Frank Drummer</h4>
<p>['drummer', 'frank', 'encyclopedia', 'britannica', 'end', 'memorize', 'purpose', 'cell', 'fool', 'space']</p>

In [27]:
similar_topics, similarity=topic_model.find_topics("encyclopedia britannica", top_n=5)
topic_model.get_topic(similar_topics[0])

[('write', 0.0932537483783199),
 ('read', 0.08752642616979708),
 ('letter', 0.07997783889252948),
 ('books', 0.07721131081461165),
 ('elected', 0.05512701828908658),
 ('legislature', 0.044311863886052086),
 ('letters', 0.044311863886052086),
 ('wrote', 0.042713303240251285),
 ('studied', 0.041345263716814934),
 ('newspapers', 0.041345263716814934)]

In [28]:
similar_topics, similarity=topic_model.find_topics("end", top_n=5)
topic_model.get_topic(similar_topics[0])

[('away', 0.16863289697787118),
 ('ran', 0.13371917268434935),
 ('lost', 0.10743535178225573),
 ('begged', 0.08564764469044946),
 ('gone', 0.07600614105746963),
 ('year', 0.06051233313032653),
 ('dissolved', 0.049196351657794325),
 ('utterly', 0.049196351657794325),
 ('lark', 0.049196351657794325),
 ('completely', 0.049196351657794325)]

In [29]:
similar_topics, similarity=topic_model.find_topics("memorize", top_n=5)
topic_model.get_topic(similar_topics[0])

[('memory', 0.23764843556637327),
 ('memories', 0.18525949652233625),
 ('forget', 0.08409041478698674),
 ('memorize', 0.08409041478698674),
 ('page', 0.0768072307358236),
 ('stand', 0.07462421122006108),
 ('forgotten', 0.06446870200860477),
 ('remember', 0.05941210889159332),
 ('previous', 0.04830187253674352),
 ('imagine', 0.04830187253674352)]

In [30]:
similar_topics, similarity=topic_model.find_topics("purpose", top_n=5)
topic_model.get_topic(similar_topics[0])

[('life', 0.17406188847976478),
 ('reasons', 0.08138318865204959),
 ('live', 0.07300128831798496),
 ('procreate', 0.050546150964855416),
 ('hungered', 0.050546150964855416),
 ('shelter', 0.050546150964855416),
 ('meaning', 0.04307739680513323),
 ('lead', 0.04307739680513323),
 ('food', 0.040691594326024795),
 ('means', 0.040691594326024795)]

In [31]:
similar_topics, similarity=topic_model.find_topics("cell", top_n=5)
topic_model.get_topic(similar_topics[0])

[('secret', 0.3285619081922),
 ('soil', 0.2197425787042768),
 ('seed', 0.12019588510078857),
 ('air', 0.08853919798472229),
 ('draws', 0.07838936971668255),
 ('plant', 0.06680647131643543),
 ('knows', 0.05176273189284689),
 ('cried', 0.05176273189284689),
 ('mound', 0.04502716931391345),
 ('learning', 0.04502716931391345)]

In [32]:
similar_topics, similarity=topic_model.find_topics("fool", top_n=5)
topic_model.get_topic(similar_topics[0])

[('hope', 0.17381267698398348),
 ('hopes', 0.12424699089618525),
 ('promise', 0.10950826120091967),
 ('regret', 0.09068574143694648),
 ('crutch', 0.09068574143694648),
 ('confessed', 0.09068574143694648),
 ('hobbling', 0.09068574143694648),
 ('rotted', 0.052090254696488106),
 ('sins', 0.052090254696488106),
 ('broods', 0.052090254696488106)]

In [33]:
similar_topics, similarity=topic_model.find_topics("space", top_n=5)
topic_model.get_topic(similar_topics[0])

[('heaven', 0.20895250724394462),
 ('earth', 0.17017759722309309),
 ('kingdom', 0.10510884820452508),
 ('gold', 0.10180003938926556),
 ('sphere', 0.08448795380940598),
 ('worlds', 0.07446561761662537),
 ('bidding', 0.06166630417712361),
 ('realms', 0.06166630417712361),
 ('losers', 0.06166630417712361),
 ('silver', 0.05255442410226254)]

<h4 style="color:red;"> 3a - Un giudice</h4>
<p>['judge', 'dwarfs', 'court', 'height', 'five foot tall', 'attorney', 'arbiter', 'honour', 'executioner']</p>

In [56]:
similar_topics, similarity=topic_model.find_topics("judge", top_n=5)
topic_model.get_topic(similar_topics[0])

[('court', 0.15069767288365063),
 ('judge', 0.13676834171292493),
 ('county', 0.09151241063775606),
 ('house', 0.06691526266573843),
 ('somers', 0.05464324396384001),
 ('circuit', 0.05273019140695238),
 ('superintendent', 0.05273019140695238),
 ('state', 0.039547643555214285),
 ('attorney', 0.039547643555214285),
 ('board', 0.03557637401812638)]

In [67]:
similar_topics, similarity=topic_model.find_topics("dwarfs, five foot tall", top_n=5)
topic_model.get_topic(similar_topics[0])

[('children', 0.1672762760185405),
 ('child', 0.12045455209200372),
 ('birth', 0.07101227308674984),
 ('generation', 0.06555071973294488),
 ('daughters', 0.05979619899488553),
 ('two', 0.04085569715663247),
 ('born', 0.038538325043224034),
 ('sons', 0.03332359390288571),
 ('working', 0.032570231079466695),
 ('generations', 0.032570231079466695)]

In [68]:
similar_topics, similarity=topic_model.find_topics("court", top_n=5)
topic_model.get_topic(similar_topics[0])

[('court', 0.15069767288365063),
 ('judge', 0.13676834171292493),
 ('county', 0.09151241063775606),
 ('house', 0.06691526266573843),
 ('somers', 0.05464324396384001),
 ('circuit', 0.05273019140695238),
 ('superintendent', 0.05273019140695238),
 ('state', 0.039547643555214285),
 ('attorney', 0.039547643555214285),
 ('board', 0.03557637401812638)]

In [74]:
similar_topics, similarity=topic_model.find_topics("height", top_n=5)
topic_model.get_topic(similar_topics[1]) #We consider the second topic because the first one seemed to confuse "height" with "eight", seen that the
#terms retrieved were "years", "seventy", "five", etc.

[('strength', 0.2815776187702009),
 ('strong', 0.09275325688369175),
 ('weak', 0.08634976532820758),
 ('weaker', 0.07838936971668255),
 ('grew', 0.07730868428051764),
 ('old', 0.058985020164446254),
 ('giant', 0.04755404526552756),
 ('gone', 0.046376628441845875),
 ('will', 0.0452855449565805),
 ('clown', 0.04502716931391345)]

In [75]:
similar_topics, similarity=topic_model.find_topics("attorney", top_n=5)
topic_model.get_topic(similar_topics[0])

[('court', 0.15069767288365063),
 ('judge', 0.13676834171292493),
 ('county', 0.09151241063775606),
 ('house', 0.06691526266573843),
 ('somers', 0.05464324396384001),
 ('circuit', 0.05273019140695238),
 ('superintendent', 0.05273019140695238),
 ('state', 0.039547643555214285),
 ('attorney', 0.039547643555214285),
 ('board', 0.03557637401812638)]

In [76]:
similar_topics, similarity=topic_model.find_topics("arbiter", top_n=5)
topic_model.get_topic(similar_topics[0])

[('hog', 0.3143349591103823),
 ('eyed', 0.24743432494298948),
 ('allen', 0.17124827409627721),
 ('boar', 0.10649742076815878),
 ('hogs', 0.10649742076815878),
 ('stealing', 0.06257614925766838),
 ('grip', 0.04464878973984695),
 ('glistened', 0.04464878973984695),
 ('pen', 0.04464878973984695),
 ('higbee', 0.04464878973984695)]

In [77]:
similar_topics, similarity=topic_model.find_topics("honour", top_n=5)
topic_model.get_topic(similar_topics[0])

[('pride', 0.3714838862557263),
 ('courage', 0.13201242782719683),
 ('hide', 0.12317443148967783),
 ('proud', 0.11080558157728947),
 ('wounded', 0.11080558157728947),
 ('heroism', 0.08301884342252792),
 ('coat', 0.08301884342252792),
 ('constancy', 0.08301884342252792),
 ('patience', 0.08301884342252792),
 ('owner', 0.08301884342252792)]

In [78]:
similar_topics, similarity=topic_model.find_topics("executioner", top_n=5)
topic_model.get_topic(similar_topics[0])

[('killed', 0.2516639715232115),
 ('captured', 0.21663577899847683),
 ('tormented', 0.1362360507446612),
 ('punished', 0.1362360507446612),
 ('murderer', 0.1362360507446612),
 ('hunt', 0.1362360507446612),
 ('experiment', 0.1362360507446612),
 ('murdered', 0.1362360507446612),
 ('unwittingly', 0.1362360507446612),
 ('brutal', 0.1362360507446612)]

<h4 style="color:red;"> 3b - Judge Selah Lively</h4>
<p>['judge', 'law', 'attorney', 'lively', 'honor', 'clerk', 'jefferson', 'probate court', 'diligence']</p>

In [45]:
similar_topics, similarity=topic_model.find_topics("judge", top_n=5)
topic_model.get_topic(similar_topics[0])

[('court', 0.15069767288365063),
 ('judge', 0.13676834171292493),
 ('county', 0.09151241063775606),
 ('house', 0.06691526266573843),
 ('somers', 0.05464324396384001),
 ('circuit', 0.05273019140695238),
 ('superintendent', 0.05273019140695238),
 ('state', 0.039547643555214285),
 ('attorney', 0.039547643555214285),
 ('board', 0.03557637401812638)]

In [53]:
similar_topics, similarity=topic_model.find_topics("probate court", top_n=5)
topic_model.get_topic(similar_topics[0])

[('court', 0.15069767288365063),
 ('judge', 0.13676834171292493),
 ('county', 0.09151241063775606),
 ('house', 0.06691526266573843),
 ('somers', 0.05464324396384001),
 ('circuit', 0.05273019140695238),
 ('superintendent', 0.05273019140695238),
 ('state', 0.039547643555214285),
 ('attorney', 0.039547643555214285),
 ('board', 0.03557637401812638)]

In [47]:
similar_topics, similarity=topic_model.find_topics("law", top_n=5)
topic_model.get_topic(similar_topics[0])

[('court', 0.15069767288365063),
 ('judge', 0.13676834171292493),
 ('county', 0.09151241063775606),
 ('house', 0.06691526266573843),
 ('somers', 0.05464324396384001),
 ('circuit', 0.05273019140695238),
 ('superintendent', 0.05273019140695238),
 ('state', 0.039547643555214285),
 ('attorney', 0.039547643555214285),
 ('board', 0.03557637401812638)]

In [48]:
similar_topics, similarity=topic_model.find_topics("attorney", top_n=5)
topic_model.get_topic(similar_topics[0])

[('court', 0.15069767288365063),
 ('judge', 0.13676834171292493),
 ('county', 0.09151241063775606),
 ('house', 0.06691526266573843),
 ('somers', 0.05464324396384001),
 ('circuit', 0.05273019140695238),
 ('superintendent', 0.05273019140695238),
 ('state', 0.039547643555214285),
 ('attorney', 0.039547643555214285),
 ('board', 0.03557637401812638)]

In [49]:
similar_topics, similarity=topic_model.find_topics("honor", top_n=5)
topic_model.get_topic(similar_topics[0])

[('pride', 0.3714838862557263),
 ('courage', 0.13201242782719683),
 ('hide', 0.12317443148967783),
 ('proud', 0.11080558157728947),
 ('wounded', 0.11080558157728947),
 ('heroism', 0.08301884342252792),
 ('coat', 0.08301884342252792),
 ('constancy', 0.08301884342252792),
 ('patience', 0.08301884342252792),
 ('owner', 0.08301884342252792)]

In [52]:
similar_topics, similarity=topic_model.find_topics("clerk", top_n=5)
topic_model.get_topic(similar_topics[0])

[('court', 0.15069767288365063),
 ('judge', 0.13676834171292493),
 ('county', 0.09151241063775606),
 ('house', 0.06691526266573843),
 ('somers', 0.05464324396384001),
 ('circuit', 0.05273019140695238),
 ('superintendent', 0.05273019140695238),
 ('state', 0.039547643555214285),
 ('attorney', 0.039547643555214285),
 ('board', 0.03557637401812638)]

In [55]:
similar_topics, similarity=topic_model.find_topics("diligence", top_n=5)
topic_model.get_topic(similar_topics[0])

[('wisdom', 0.44653925783591225),
 ('numbers', 0.07278364354851763),
 ('intelligence', 0.07278364354851763),
 ('magic', 0.07278364354851763),
 ('pope', 0.07278364354851763),
 ('sealing', 0.07278364354851763),
 ('dispel', 0.07278364354851763),
 ('witchcraft', 0.07278364354851763),
 ('brotherhood', 0.07278364354851763),
 ('material', 0.07278364354851763)]

<h4 style="color:red;"> 4a - Un blasfemo</h4>
<p>['blasphemer', 'enchanted garden', 'god', 'forbidden apple', 'flower', 'punishing', 'soul', 'death', 'obliges']</p>

In [96]:
similar_topics, similarity=topic_model.find_topics("blasphemer", top_n=5)
topic_model.get_topic(similar_topics[0])

[('jesus', 0.3032521520454909),
 ('thou', 0.17505353512701602),
 ('unto', 0.16517760047443822),
 ('confucius', 0.16517760047443822),
 ('repent', 0.09487867819717477),
 ('shalt', 0.09487867819717477),
 ('statute', 0.09487867819717477),
 ('blasphemy', 0.09487867819717477),
 ('canst', 0.09487867819717477),
 ('thee', 0.09487867819717477)]

In [99]:
similar_topics, similarity=topic_model.find_topics("enchanted garden, flower", top_n=5)
topic_model.get_topic(similar_topics[0])

[('bouquet', 0.1006292129954397),
 ('clover', 0.1006292129954397),
 ('blossoms', 0.08522339043610141),
 ('flowers', 0.08050337039635176),
 ('flower', 0.06850374633195079),
 ('weed', 0.06391754282707607),
 ('petals', 0.049999706089559684),
 ('fragrant', 0.049999706089559684),
 ('roses', 0.045669164221300525),
 ('flowering', 0.045669164221300525)]

In [100]:
similar_topics, similarity=topic_model.find_topics("god", top_n=5)
topic_model.get_topic(similar_topics[0])

[('cursed', 0.32036723166605996),
 ('god', 0.21222464353272202),
 ('chose', 0.11417291055325132),
 ('curse', 0.10652923804512678),
 ('destined', 0.07180008079786199),
 ('genesis', 0.07180008079786199),
 ('sex', 0.07180008079786199),
 ('familiarly', 0.07180008079786199),
 ('alienated', 0.07180008079786199),
 ('damned', 0.07180008079786199)]

In [101]:
similar_topics, similarity=topic_model.find_topics("forbidden apple", top_n=5)
topic_model.get_topic(similar_topics[0])

[('cursed', 0.32036723166605996),
 ('god', 0.21222464353272202),
 ('chose', 0.11417291055325132),
 ('curse', 0.10652923804512678),
 ('destined', 0.07180008079786199),
 ('genesis', 0.07180008079786199),
 ('sex', 0.07180008079786199),
 ('familiarly', 0.07180008079786199),
 ('alienated', 0.07180008079786199),
 ('damned', 0.07180008079786199)]

In [102]:
similar_topics, similarity=topic_model.find_topics("punishing", top_n=5)
topic_model.get_topic(similar_topics[0])

[('killed', 0.2516639715232115),
 ('captured', 0.21663577899847683),
 ('tormented', 0.1362360507446612),
 ('punished', 0.1362360507446612),
 ('murderer', 0.1362360507446612),
 ('hunt', 0.1362360507446612),
 ('experiment', 0.1362360507446612),
 ('murdered', 0.1362360507446612),
 ('unwittingly', 0.1362360507446612),
 ('brutal', 0.1362360507446612)]

In [105]:
similar_topics, similarity=topic_model.find_topics("soul", top_n=5)
topic_model.get_topic(similar_topics[0])

[('soul', 0.21204952130363175),
 ('souls', 0.09655057373443228),
 ('your', 0.042519229082855665),
 ('body', 0.02936539597767276),
 ('cured', 0.02842486880531027),
 ('snared', 0.02223544621771284),
 ('miles', 0.01632896203321874),
 ('beloved', 0.01632896203321874),
 ('truth', 0.016171739198973606),
 ('makes', 0.01468269798883638)]

In [103]:
similar_topics, similarity=topic_model.find_topics("death", top_n=5)
topic_model.get_topic(similar_topics[0])

[('grave', 0.15679348639741028),
 ('dead', 0.13750869138150082),
 ('died', 0.06139162958986042),
 ('alive', 0.053911493696976155),
 ('cemetery', 0.0489415112516854),
 ('graven', 0.0489415112516854),
 ('funeral', 0.0489415112516854),
 ('buried', 0.04470262106317776),
 ('bring', 0.035940995797984106),
 ('shame', 0.035940995797984106)]

In [104]:
similar_topics, similarity=topic_model.find_topics("obliges", top_n=5)
topic_model.get_topic(similar_topics[0])

[('people', 0.08289938385658228),
 ('democracy', 0.06569303012782818),
 ('republic', 0.062054681347187814),
 ('liberty', 0.062054681347187814),
 ('poverty', 0.05280497113087873),
 ('defeat', 0.049269772595871134),
 ('liberals', 0.04245530128636873),
 ('freedom', 0.04245530128636873),
 ('enemy', 0.038541440110702256),
 ('chiefly', 0.038541440110702256)]

<h4 style="color:red;"> 4b - Wendell P. Bloyd</h4>
<p>['genesis', 'catholic', 'god', 'adam', 'crucified', 'blasphemy', 'evil', 'wretched', 'death', 'eden']</p>

In [90]:
similar_topics, similarity=topic_model.find_topics("genesis, catholic, god, adam, crucified, genesis", top_n=5)
topic_model.get_topic(similar_topics[0])

[('church', 0.16904783085158634),
 ('faith', 0.0543414346432684),
 ('christian', 0.05007917224578747),
 ('presbyterian', 0.04855629529276205),
 ('congregation', 0.04855629529276205),
 ('bell', 0.04530553801919184),
 ('sunday', 0.0427963319635778),
 ('preached', 0.035440404699496324),
 ('matheny', 0.035440404699496324),
 ('prayer', 0.035440404699496324)]

In [93]:
similar_topics, similarity=topic_model.find_topics("evil, blasphemy", top_n=5)
topic_model.get_topic(similar_topics[0])

[('cursed', 0.32036723166605996),
 ('god', 0.21222464353272202),
 ('chose', 0.11417291055325132),
 ('curse', 0.10652923804512678),
 ('destined', 0.07180008079786199),
 ('genesis', 0.07180008079786199),
 ('sex', 0.07180008079786199),
 ('familiarly', 0.07180008079786199),
 ('alienated', 0.07180008079786199),
 ('damned', 0.07180008079786199)]

In [94]:
similar_topics, similarity=topic_model.find_topics("wretched", top_n=5)
topic_model.get_topic(similar_topics[0])

[('sorrow', 0.3377408603472646),
 ('despair', 0.08864954478169687),
 ('divine', 0.053911493696976155),
 ('hear', 0.043432080604268355),
 ('supremer', 0.042168301420966565),
 ('serene', 0.042168301420966565),
 ('sad', 0.042168301420966565),
 ('ours', 0.042168301420966565),
 ('ovid', 0.042168301420966565),
 ('itys', 0.042168301420966565)]

In [95]:
similar_topics, similarity=topic_model.find_topics("death", top_n=5)
topic_model.get_topic(similar_topics[0])

[('grave', 0.15679348639741028),
 ('dead', 0.13750869138150082),
 ('died', 0.06139162958986042),
 ('alive', 0.053911493696976155),
 ('cemetery', 0.0489415112516854),
 ('graven', 0.0489415112516854),
 ('funeral', 0.0489415112516854),
 ('buried', 0.04470262106317776),
 ('bring', 0.035940995797984106),
 ('shame', 0.035940995797984106)]

<h4 style="color:red;"> 5a - Un malato di cuore</h4>
<p>['heart patient', 'heart', 'soul', 'caresses', 'flower', 'feel', 'narrating', 'breath', 'feeling', 'rhythm']</p>

In [116]:
similar_topics, similarity=topic_model.find_topics("heart patient", top_n=5)
topic_model.get_topic(similar_topics[1])

[('fever', 0.06433092929650201),
 ('lungs', 0.06433092929650201),
 ('breathe', 0.06433092929650201),
 ('worried', 0.06002408844167042),
 ('sick', 0.05849887036007819),
 ('lips', 0.056699708845146225),
 ('feverish', 0.04695403871354589),
 ('aching', 0.04695403871354589),
 ('kiss', 0.042887286197668004),
 ('sickness', 0.042887286197668004)]

In [118]:
similar_topics, similarity=topic_model.find_topics("heart", top_n=5)
topic_model.get_topic(similar_topics[1])

[('soul', 0.21204952130363175),
 ('souls', 0.09655057373443228),
 ('your', 0.042519229082855665),
 ('body', 0.02936539597767276),
 ('cured', 0.02842486880531027),
 ('snared', 0.02223544621771284),
 ('miles', 0.01632896203321874),
 ('beloved', 0.01632896203321874),
 ('truth', 0.016171739198973606),
 ('makes', 0.01468269798883638)]

In [120]:
similar_topics, similarity=topic_model.find_topics("soul", top_n=5)
topic_model.get_topic(similar_topics[0])

[('soul', 0.21204952130363175),
 ('souls', 0.09655057373443228),
 ('your', 0.042519229082855665),
 ('body', 0.02936539597767276),
 ('cured', 0.02842486880531027),
 ('snared', 0.02223544621771284),
 ('miles', 0.01632896203321874),
 ('beloved', 0.01632896203321874),
 ('truth', 0.016171739198973606),
 ('makes', 0.01468269798883638)]

In [123]:
similar_topics, similarity=topic_model.find_topics("caresses", top_n=5)
topic_model.get_topic(similar_topics[0])

[('hand', 0.20815807144894904),
 ('hands', 0.12929870416585848),
 ('arms', 0.07347265023746416),
 ('giant', 0.04637501934985332),
 ('grasp', 0.0382229158122667),
 ('observe', 0.0382229158122667),
 ('arm', 0.03491237760719255),
 ('helped', 0.03491237760719255),
 ('steel', 0.03491237760719255),
 ('flung', 0.030770916370506354)]

In [125]:
similar_topics, similarity=topic_model.find_topics("flower", top_n=5)
topic_model.get_topic(similar_topics[0])

[('bouquet', 0.1006292129954397),
 ('clover', 0.1006292129954397),
 ('blossoms', 0.08522339043610141),
 ('flowers', 0.08050337039635176),
 ('flower', 0.06850374633195079),
 ('weed', 0.06391754282707607),
 ('petals', 0.049999706089559684),
 ('fragrant', 0.049999706089559684),
 ('roses', 0.045669164221300525),
 ('flowering', 0.045669164221300525)]

In [126]:
similar_topics, similarity=topic_model.find_topics("feel, feeling", top_n=5)
topic_model.get_topic(similar_topics[0])

[('tears', 0.19755979286073697),
 ('laughed', 0.10744783668100798),
 ('cries', 0.10744783668100798),
 ('crying', 0.08595826934480637),
 ('laugh', 0.07166512377581255),
 ('wept', 0.05606027652465782),
 ('cry', 0.047776749183875034),
 ('broken', 0.042957260613394464),
 ('heard', 0.038237982498664815),
 ('cried', 0.037018196141551114)]

In [128]:
similar_topics, similarity=topic_model.find_topics("narrating", top_n=5)
topic_model.get_topic(similar_topics[0])

[('silence', 0.156885717859997),
 ('speak', 0.0961078232069892),
 ('whispers', 0.08343008495231206),
 ('talking', 0.06955732601770041),
 ('silent', 0.05993689593369702),
 ('stood', 0.057734675820030876),
 ('heard', 0.05567000393187965),
 ('spoke', 0.05567000393187965),
 ('facing', 0.05441144486216789),
 ('witness', 0.0496987963584741)]

In [129]:
similar_topics, similarity=topic_model.find_topics("breath", top_n=5)
topic_model.get_topic(similar_topics[0])

[('fever', 0.06433092929650201),
 ('lungs', 0.06433092929650201),
 ('breathe', 0.06433092929650201),
 ('worried', 0.06002408844167042),
 ('sick', 0.05849887036007819),
 ('lips', 0.056699708845146225),
 ('feverish', 0.04695403871354589),
 ('aching', 0.04695403871354589),
 ('kiss', 0.042887286197668004),
 ('sickness', 0.042887286197668004)]

In [131]:
similar_topics, similarity=topic_model.find_topics("rhytm", top_n=5)
topic_model.get_topic(similar_topics[0])

[('clang', 0.3249536684977153),
 ('devour', 0.2864062216024053),
 ('acorns', 0.23717809298893694),
 ('poisoned', 0.20213240039331745),
 ('acorn', 0.1362360507446612),
 ('retaliation', 0.1362360507446612),
 ('lovely', 0.1362360507446612),
 ('exploded', 0.1362360507446612),
 ('dropping', 0.1362360507446612),
 ('devourer', 0.1362360507446612)]

<h4 style="color:red;"> 5b - Francis Turner</h4>
<p>['boyhood', 'turner', 'francis', 'mary', 'garden', 'manhood', 'scarlet fever', 'soul', 'heart diseased']</p>

In [106]:
similar_topics, similarity=topic_model.find_topics("boyhood", top_n=5)
topic_model.get_topic(similar_topics[0])

[('boy', 0.2967989782399862),
 ('girls', 0.1997896531123234),
 ('boyhood', 0.13602861215541973),
 ('boys', 0.13073809821666418),
 ('wanton', 0.12424699089618525),
 ('sent', 0.0961078232069892),
 ('arbor', 0.07813538204473217),
 ('ann', 0.07813538204473217),
 ('balloons', 0.07813538204473217),
 ('reckless', 0.07813538204473217)]

In [107]:
similar_topics, similarity=topic_model.find_topics("garden", top_n=5)
topic_model.get_topic(similar_topics[0])

[('bouquet', 0.1006292129954397),
 ('clover', 0.1006292129954397),
 ('blossoms', 0.08522339043610141),
 ('flowers', 0.08050337039635176),
 ('flower', 0.06850374633195079),
 ('weed', 0.06391754282707607),
 ('petals', 0.049999706089559684),
 ('fragrant', 0.049999706089559684),
 ('roses', 0.045669164221300525),
 ('flowering', 0.045669164221300525)]

In [108]:
similar_topics, similarity=topic_model.find_topics("manhood", top_n=5)
topic_model.get_topic(similar_topics[0])

[('men', 0.252704975849759),
 ('women', 0.18965740114630747),
 ('woman', 0.09450597334946784),
 ('other', 0.08010418523059828),
 ('neither', 0.04741435028657687),
 ('are', 0.04649931072982677),
 ('white', 0.0437795372491025),
 ('jokes', 0.0425056478323343),
 ('babylon', 0.0425056478323343),
 ('endowed', 0.0425056478323343)]

In [109]:
similar_topics, similarity=topic_model.find_topics("scarlet fever", top_n=5)
topic_model.get_topic(similar_topics[0])

[('fever', 0.06433092929650201),
 ('lungs', 0.06433092929650201),
 ('breathe', 0.06433092929650201),
 ('worried', 0.06002408844167042),
 ('sick', 0.05849887036007819),
 ('lips', 0.056699708845146225),
 ('feverish', 0.04695403871354589),
 ('aching', 0.04695403871354589),
 ('kiss', 0.042887286197668004),
 ('sickness', 0.042887286197668004)]

In [110]:
similar_topics, similarity=topic_model.find_topics("soul", top_n=5)
topic_model.get_topic(similar_topics[0])

[('soul', 0.21204952130363175),
 ('souls', 0.09655057373443228),
 ('your', 0.042519229082855665),
 ('body', 0.02936539597767276),
 ('cured', 0.02842486880531027),
 ('snared', 0.02223544621771284),
 ('miles', 0.01632896203321874),
 ('beloved', 0.01632896203321874),
 ('truth', 0.016171739198973606),
 ('makes', 0.01468269798883638)]

In [113]:
similar_topics, similarity=topic_model.find_topics("heart diseased", top_n=5)
topic_model.get_topic(similar_topics[1])

[('fever', 0.06433092929650201),
 ('lungs', 0.06433092929650201),
 ('breathe', 0.06433092929650201),
 ('worried', 0.06002408844167042),
 ('sick', 0.05849887036007819),
 ('lips', 0.056699708845146225),
 ('feverish', 0.04695403871354589),
 ('aching', 0.04695403871354589),
 ('kiss', 0.042887286197668004),
 ('sickness', 0.042887286197668004)]

<h4 style="color:red;"> 6a - Un dottore</h4>
<p>['doctor', 'flowers', 'cherries', 'illness', 'cure', 'diagnosis', 'health', 'children', 'child']</p>

In [201]:
similar_topics, similarity=topic_model.find_topics("doctor, cure, diagnosis, illness, health", top_n=5)
topic_model.get_topic(similar_topics[0])

[('fever', 0.06433092929650201),
 ('lungs', 0.06433092929650201),
 ('breathe', 0.06433092929650201),
 ('worried', 0.06002408844167042),
 ('sick', 0.05849887036007819),
 ('lips', 0.056699708845146225),
 ('feverish', 0.04695403871354589),
 ('aching', 0.04695403871354589),
 ('kiss', 0.042887286197668004),
 ('sickness', 0.042887286197668004)]

In [202]:
similar_topics, similarity=topic_model.find_topics("flowers, cherries", top_n=5)
topic_model.get_topic(similar_topics[0])

[('bouquet', 0.1006292129954397),
 ('clover', 0.1006292129954397),
 ('blossoms', 0.08522339043610141),
 ('flowers', 0.08050337039635176),
 ('flower', 0.06850374633195079),
 ('weed', 0.06391754282707607),
 ('petals', 0.049999706089559684),
 ('fragrant', 0.049999706089559684),
 ('roses', 0.045669164221300525),
 ('flowering', 0.045669164221300525)]

In [203]:
similar_topics, similarity=topic_model.find_topics("children, child", top_n=5)
topic_model.get_topic(similar_topics[0])

[('children', 0.1672762760185405),
 ('child', 0.12045455209200372),
 ('birth', 0.07101227308674984),
 ('generation', 0.06555071973294488),
 ('daughters', 0.05979619899488553),
 ('two', 0.04085569715663247),
 ('born', 0.038538325043224034),
 ('sons', 0.03332359390288571),
 ('working', 0.032570231079466695),
 ('generations', 0.032570231079466695)]

<h4 style="color:red;"> 6b - Dr Siegfried Iseman</h4>
<p>['doctor', 'dr', 'doctors', 'medicine', 'iseman', 'swindler and crook', 'diploma', 'christian', 'youth', 'siegfried']</p>

In [173]:
similar_topics, similarity=topic_model.find_topics(" doctor, doctors, medicine, diploma", top_n=5)
topic_model.get_topic(similar_topics[0])

[('write', 0.0932537483783199),
 ('read', 0.08752642616979708),
 ('letter', 0.07997783889252948),
 ('books', 0.07721131081461165),
 ('elected', 0.05512701828908658),
 ('legislature', 0.044311863886052086),
 ('letters', 0.044311863886052086),
 ('wrote', 0.042713303240251285),
 ('studied', 0.041345263716814934),
 ('newspapers', 0.041345263716814934)]

In [165]:
similar_topics, similarity=topic_model.find_topics("swindler and crook", top_n=5)
topic_model.get_topic(similar_topics[1])

[('thief', 0.28749556301134566),
 ('stole', 0.15979385706769017),
 ('theft', 0.12499926522389922),
 ('steal', 0.11417291055325132),
 ('apple', 0.10652923804512678),
 ('stealing', 0.1006292129954397),
 ('proctor', 0.07180008079786199),
 ('husbands', 0.07180008079786199),
 ('regard', 0.07180008079786199),
 ('prestige', 0.07180008079786199)]

In [166]:
similar_topics, similarity=topic_model.find_topics("christian", top_n=5)
topic_model.get_topic(similar_topics[0])

[('church', 0.16904783085158634),
 ('faith', 0.0543414346432684),
 ('christian', 0.05007917224578747),
 ('presbyterian', 0.04855629529276205),
 ('congregation', 0.04855629529276205),
 ('bell', 0.04530553801919184),
 ('sunday', 0.0427963319635778),
 ('preached', 0.035440404699496324),
 ('matheny', 0.035440404699496324),
 ('prayer', 0.035440404699496324)]

In [179]:
similar_topics, similarity=topic_model.find_topics("youth", top_n=5)
topic_model.get_topic(similar_topics[0])

[('children', 0.1672762760185405),
 ('child', 0.12045455209200372),
 ('birth', 0.07101227308674984),
 ('generation', 0.06555071973294488),
 ('daughters', 0.05979619899488553),
 ('two', 0.04085569715663247),
 ('born', 0.038538325043224034),
 ('sons', 0.03332359390288571),
 ('working', 0.032570231079466695),
 ('generations', 0.032570231079466695)]

<h4 style="color:red;"> 7a - Un chimico</h4>
<p>['chemist', 'death', 'oxygen', 'phosphorous', 'hydrogen', 'ashes', 'corpse', 'dying', 'die', 'suffering']</p>

In [220]:
similar_topics, similarity=topic_model.find_topics("chemist, oxygen, hydrogen, phosphorous", top_n=5)
topic_model.get_topic(similar_topics[0])

[('water', 0.4376263833305979),
 ('waters', 0.14015069131164457),
 ('fluids', 0.08050312089457254),
 ('pour', 0.08050312089457254),
 ('pool', 0.08050312089457254),
 ('pent', 0.08050312089457254),
 ('dozen', 0.08050312089457254),
 ('deeply', 0.08050312089457254),
 ('cleaned', 0.08050312089457254),
 ('sandy', 0.08050312089457254)]

In [223]:
similar_topics, similarity=topic_model.find_topics("corpse, ashes", top_n=5)
topic_model.get_topic(similar_topics[0])

[('grave', 0.15679348639741028),
 ('dead', 0.13750869138150082),
 ('died', 0.06139162958986042),
 ('alive', 0.053911493696976155),
 ('cemetery', 0.0489415112516854),
 ('graven', 0.0489415112516854),
 ('funeral', 0.0489415112516854),
 ('buried', 0.04470262106317776),
 ('bring', 0.035940995797984106),
 ('shame', 0.035940995797984106)]

In [227]:
similar_topics, similarity=topic_model.find_topics("dying, die", top_n=5)
topic_model.get_topic(similar_topics[0])

[('grave', 0.15679348639741028),
 ('dead', 0.13750869138150082),
 ('died', 0.06139162958986042),
 ('alive', 0.053911493696976155),
 ('cemetery', 0.0489415112516854),
 ('graven', 0.0489415112516854),
 ('funeral', 0.0489415112516854),
 ('buried', 0.04470262106317776),
 ('bring', 0.035940995797984106),
 ('shame', 0.035940995797984106)]

In [228]:
similar_topics, similarity=topic_model.find_topics("suffering", top_n=5)
topic_model.get_topic(similar_topics[0])

[('sorrow', 0.3377408603472646),
 ('despair', 0.08864954478169687),
 ('divine', 0.053911493696976155),
 ('hear', 0.043432080604268355),
 ('supremer', 0.042168301420966565),
 ('serene', 0.042168301420966565),
 ('sad', 0.042168301420966565),
 ('ours', 0.042168301420966565),
 ('ovid', 0.042168301420966565),
 ('itys', 0.042168301420966565)]

<h4 style="color:red;"> 7b - Trainor the druggist</h4>
<p>['chemist', 'druggist',  'trainor', 'compounding', 'hydrogen', 'experiment', 'mixer of chemicals', 'pantier', 'fluids']</p>

In [212]:
similar_topics, similarity=topic_model.find_topics("chemist, druggist, compoundig fluids, hydrogen, mixer of chemicals, experiment", top_n=5)
topic_model.get_topic(similar_topics[0])

[('water', 0.4376263833305979),
 ('waters', 0.14015069131164457),
 ('fluids', 0.08050312089457254),
 ('pour', 0.08050312089457254),
 ('pool', 0.08050312089457254),
 ('pent', 0.08050312089457254),
 ('dozen', 0.08050312089457254),
 ('deeply', 0.08050312089457254),
 ('cleaned', 0.08050312089457254),
 ('sandy', 0.08050312089457254)]

<h4 style="color:red;"> 8a - Un ottico</h4>
<p>['optician', 'glasses', 'sight', 'eyes', 'sighted', 'lens', 'pupils', 'light', 'blind', 'glance']</p>

In [256]:
similar_topics, similarity=topic_model.find_topics("optician, glasses, sight, eyes, sighted, lens, pupils, blind, glance ", top_n=5)
topic_model.get_topic(similar_topics[0])

[('eyes', 0.1344257247519203),
 ('eye', 0.1196434702965231),
 ('vision', 0.06587515232779464),
 ('wide', 0.031103321144502306),
 ('look', 0.0286985230684364),
 ('haggard', 0.0270466246390893),
 ('downcast', 0.0270466246390893),
 ('lashes', 0.0270466246390893),
 ('his', 0.025636494175516722),
 ('staring', 0.024704080061229813)]

In [253]:
similar_topics, similarity=topic_model.find_topics("light", top_n=5)
topic_model.get_topic(similar_topics[0])

[('darkness', 0.3087658275372271),
 ('light', 0.15720620307085756),
 ('bellowing', 0.1280120512263727),
 ('spear', 0.08050312089457254),
 ('staggered', 0.08050312089457254),
 ('lantern', 0.08050312089457254),
 ('plate', 0.08050312089457254),
 ('fantastic', 0.08050312089457254),
 ('skipping', 0.08050312089457254),
 ('splinters', 0.08050312089457254)]

<h4 style="color:red;"> 8b - Dippold the optician</h4>
<p>['dippold', 'optician', 'glasses', 'lens', 'eyes', 'globes', 'light', 'angels bending', 'depths of air']</p>

In [234]:
similar_topics, similarity=topic_model.find_topics("optician, glasses, lens, eyes ", top_n=5)
topic_model.get_topic(similar_topics[0])

[('eyes', 0.1344257247519203),
 ('eye', 0.1196434702965231),
 ('vision', 0.06587515232779464),
 ('wide', 0.031103321144502306),
 ('look', 0.0286985230684364),
 ('haggard', 0.0270466246390893),
 ('downcast', 0.0270466246390893),
 ('lashes', 0.0270466246390893),
 ('his', 0.025636494175516722),
 ('staring', 0.024704080061229813)]

In [233]:
similar_topics, similarity=topic_model.find_topics("globes", top_n=5)
topic_model.get_topic(similar_topics[0])

[('heaven', 0.20895250724394462),
 ('earth', 0.17017759722309309),
 ('kingdom', 0.10510884820452508),
 ('gold', 0.10180003938926556),
 ('sphere', 0.08448795380940598),
 ('worlds', 0.07446561761662537),
 ('bidding', 0.06166630417712361),
 ('realms', 0.06166630417712361),
 ('losers', 0.06166630417712361),
 ('silver', 0.05255442410226254)]

In [235]:
similar_topics, similarity=topic_model.find_topics("light", top_n=5)
topic_model.get_topic(similar_topics[0])

[('darkness', 0.3087658275372271),
 ('light', 0.15720620307085756),
 ('bellowing', 0.1280120512263727),
 ('spear', 0.08050312089457254),
 ('staggered', 0.08050312089457254),
 ('lantern', 0.08050312089457254),
 ('plate', 0.08050312089457254),
 ('fantastic', 0.08050312089457254),
 ('skipping', 0.08050312089457254),
 ('splinters', 0.08050312089457254)]

In [240]:
similar_topics, similarity=topic_model.find_topics("angels bending", top_n=5)
topic_model.get_topic(similar_topics[1])

[('spirit', 0.37312947722132966),
 ('spirits', 0.2628861296783206),
 ('aflame', 0.06325245213144984),
 ('gilt', 0.06325245213144984),
 ('spiritus', 0.06325245213144984),
 ('swarmed', 0.06325245213144984),
 ('frumenti', 0.06325245213144984),
 ('audacity', 0.06325245213144984),
 ('girded', 0.06325245213144984),
 ('vanquished', 0.06325245213144984)]

In [242]:
similar_topics, similarity=topic_model.find_topics("depths of air", top_n=5)
topic_model.get_topic(similar_topics[0])

[('wind', 0.3040245642298785),
 ('winds', 0.1877510084653466),
 ('blew', 0.1877510084653466),
 ('kite', 0.17518141367420847),
 ('tumbling', 0.11807124397870639),
 ('blown', 0.11807124397870639),
 ('pockets', 0.11807124397870639),
 ('pasture', 0.11807124397870639),
 ('pane', 0.11807124397870639),
 ('northwest', 0.11807124397870639)]

<h4 style="color:red;"> 9a - Il suonatore Jones</h4>
<p>['jones', 'flute', 'play', 'playing', 'player', 'dust', 'played', 'drought', 'vibrate with sounds', 'listened']</p>

In [286]:
similar_topics, similarity=topic_model.find_topics("play, playing, player, played, flute, listened", top_n=5)
topic_model.get_topic(similar_topics[0])

[('voice', 0.08733661231645529),
 ('song', 0.0689840196590129),
 ('music', 0.0660782899398735),
 ('ear', 0.05794989697791858),
 ('singing', 0.0528626319518988),
 ('sound', 0.050858614615371715),
 ('ears', 0.04931203529731866),
 ('rhythm', 0.04931203529731866),
 ('trumpets', 0.04931203529731866),
 ('round', 0.0396469739639241)]

In [280]:
similar_topics, similarity=topic_model.find_topics("dust", top_n=5)
topic_model.get_topic(similar_topics[0])

[('dust', 0.5518111987076002),
 ('mud', 0.13028092431786678),
 ('howls', 0.07483388702875757),
 ('protest', 0.07483388702875757),
 ('portals', 0.07483388702875757),
 ('triumphant', 0.07483388702875757),
 ('brings', 0.07483388702875757),
 ('box', 0.07483388702875757),
 ('career', 0.07483388702875757),
 ('hazed', 0.07483388702875757)]

In [281]:
similar_topics, similarity=topic_model.find_topics("drought", top_n=5)
topic_model.get_topic(similar_topics[0])

[('water', 0.4376263833305979),
 ('waters', 0.14015069131164457),
 ('fluids', 0.08050312089457254),
 ('pour', 0.08050312089457254),
 ('pool', 0.08050312089457254),
 ('pent', 0.08050312089457254),
 ('dozen', 0.08050312089457254),
 ('deeply', 0.08050312089457254),
 ('cleaned', 0.08050312089457254),
 ('sandy', 0.08050312089457254)]

In [284]:
similar_topics, similarity=topic_model.find_topics("vibrate with sounds", top_n=5)
topic_model.get_topic(similar_topics[1])

[('dance', 0.34474822970659896),
 ('danced', 0.2919690227903474),
 ('rigadoon', 0.09839270331558865),
 ('viols', 0.09839270331558865),
 ('picnic', 0.09839270331558865),
 ('chandlerville', 0.09839270331558865),
 ('platter', 0.09839270331558865),
 ('dances', 0.09839270331558865),
 ('merrily', 0.09839270331558865),
 ('jocund', 0.09839270331558865)]

<h4 style="color:red;"> 9b - Fiddler Jones</h4>
<p>['fiddler', 'fiddle', 'jones', 'whirling', 'potter', 'bassoons', 'meadow', 'acres', 'rustle', 'cooney']</p>

In [261]:
similar_topics, similarity=topic_model.find_topics("fiddle, fiddler", top_n=5)
topic_model.get_topic(similar_topics[0])

[('fiddle', 0.41715042476156033),
 ('fiddler', 0.34762535396796695),
 ('jones', 0.22596622472422362),
 ('fiddlers', 0.10418050939297621),
 ('lowest', 0.10418050939297621),
 ('fiddled', 0.10418050939297621),
 ('fair', 0.09068574143694648),
 ('lack', 0.09068574143694648),
 ('roaring', 0.09068574143694648),
 ('mr', 0.0828313272641235)]

In [275]:
similar_topics, similarity=topic_model.find_topics("whirling, rustle", top_n=5)
topic_model.get_topic(similar_topics[0])

[('wind', 0.3040245642298785),
 ('winds', 0.1877510084653466),
 ('blew', 0.1877510084653466),
 ('kite', 0.17518141367420847),
 ('tumbling', 0.11807124397870639),
 ('blown', 0.11807124397870639),
 ('pockets', 0.11807124397870639),
 ('pasture', 0.11807124397870639),
 ('pane', 0.11807124397870639),
 ('northwest', 0.11807124397870639)]

In [264]:
similar_topics, similarity=topic_model.find_topics("meadow", top_n=5)
topic_model.get_topic(similar_topics[0])

[('hill', 0.09884535193548062),
 ('mountains', 0.08710678027999316),
 ('hills', 0.08186179262185232),
 ('mountain', 0.06533008520999486),
 ('meadow', 0.06171183780383318),
 ('golden', 0.05228674402441735),
 ('slopes', 0.05110467196999194),
 ('valleys', 0.05110467196999194),
 ('plow', 0.046678427519008824),
 ('lawn', 0.04355339013999658)]

In [265]:
similar_topics, similarity=topic_model.find_topics("bassoons", top_n=5)
topic_model.get_topic(similar_topics[0])

[('voice', 0.08733661231645529),
 ('song', 0.0689840196590129),
 ('music', 0.0660782899398735),
 ('ear', 0.05794989697791858),
 ('singing', 0.0528626319518988),
 ('sound', 0.050858614615371715),
 ('ears', 0.04931203529731866),
 ('rhythm', 0.04931203529731866),
 ('trumpets', 0.04931203529731866),
 ('round', 0.0396469739639241)]

In [266]:
similar_topics, similarity=topic_model.find_topics("acres", top_n=5)
topic_model.get_topic(similar_topics[0])

[('farm', 0.3505148395710941),
 ('mortgaged', 0.11734438029084161),
 ('fretting', 0.07379452748669148),
 ('frosts', 0.07379452748669148),
 ('toothless', 0.07379452748669148),
 ('juan', 0.07379452748669148),
 ('rural', 0.07379452748669148),
 ('corporation', 0.07379452748669148),
 ('crops', 0.07379452748669148),
 ('goats', 0.07379452748669148)]